In [ ]:
## This file implements neural networks for p0017SkpresabsSTCC_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for four replicates.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p0017SkpresabsSTCC_qual.csv')
df.shape

(253, 105)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
      ..
248    1
249    0
250    0
251    0
252    0
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT,TTTTAATACATAT,TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC,TTCCATCGAATCAC,TTCATTTAATGGCTAAGGAAATTGTGCGATTCCACTCAATTATTTGGCCTATTTTATTGATGGCATTAGACTTACCGTTACCTAAAAAAGTCTTTGCACA,...,AGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTACATTATTCATCT,AGATATTCATTTAATGGCTAAGGAAATTGTGCGATTCCACTCAATTATTTGGCCTATTTTATTGATGGCATTAGACTTACCGTTACCTAAAAAAGTCTTT,ACTTTCGAATT,AATTAAAATCTTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTT,AATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAATCTTAATGCTT,AATGGTATGCCTTTGTTTGTATAGTTTTCACTTCCACCTTTGGGAGTCTTTCCACTACCTATTTTGGTAGTAAGATTCCCTAACTTCTTCTCTTCCCATT,AAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAATCTTAATGCTTCAAAAGATTT,ST,CC,pheno
0,107,1,0,1,0,0,1,1,0,1,...,1,1,0,1,1,1,1,5,5,0
1,109,1,0,1,0,0,1,1,0,1,...,1,1,0,1,1,1,1,8,8,0
2,115,1,1,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,5,5,1
3,120335,1,0,1,0,0,1,1,0,1,...,1,1,0,1,1,1,1,5,5,0
4,120337,1,0,1,0,0,1,1,0,1,...,1,1,0,1,1,1,1,5,5,0


In [6]:
df['pheno'].value_counts()

0    216
1     35
2      2
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 104)

In [9]:
df_clean.head()

,TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT,TTTTAATACATAT,TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC,TTCCATCGAATCAC,TTCATTTAATGGCTAAGGAAATTGTGCGATTCCACTCAATTATTTGGCCTATTTTATTGATGGCATTAGACTTACCGTTACCTAAAAAAGTCTTTGCACA,TTCAAGAAGGAGA,...,AGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTACATTATTCATCT,AGATATTCATTTAATGGCTAAGGAAATTGTGCGATTCCACTCAATTATTTGGCCTATTTTATTGATGGCATTAGACTTACCGTTACCTAAAAAAGTCTTT,ACTTTCGAATT,AATTAAAATCTTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTT,AATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAATCTTAATGCTT,AATGGTATGCCTTTGTTTGTATAGTTTTCACTTCCACCTTTGGGAGTCTTTCCACTACCTATTTTGGTAGTAAGATTCCCTAACTTCTTCTCTTCCCATT,AAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAATCTTAATGCTTCAAAAGATTT,ST,CC,pheno
0,1,0,1,0,0,1,1,0,1,0,...,1,1,0,1,1,1,1,5,5,0
1,1,0,1,0,0,1,1,0,1,0,...,1,1,0,1,1,1,1,8,8,0
2,1,1,1,0,1,1,1,1,1,0,...,1,1,1,1,1,1,1,5,5,1
3,1,0,1,0,0,1,1,0,1,0,...,1,1,0,1,1,1,1,5,5,0
4,1,0,1,0,0,1,1,0,1,0,...,1,1,0,1,1,1,1,5,5,0


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 104) (253,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in versi

[(0, 216), (1, 216), (2, 216)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is depreca

In [12]:
############# Fully-Connected Neural Network ################

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [16]:
dat

,0,test
0,312,1
1,CFBRSa27,0
2,BCH-SA-01,1
3,GA27,1
4,NRS209,2
...,...,...
190,NRS209,2
191,NRS235,0
192,NRS240,1
193,NRS110,2


In [17]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 315us/step - loss: 5.8264 - accuracy: 0.4393 - val_loss: 1.4900 - val_accuracy: 0.5282
Epoch 2/1000
453/453 [==============================] - 0s 108us/step - loss: 1.2586 - accuracy: 0.6026 - val_loss: 0.8574 - val_accuracy: 0.5436
Epoch 3/1000
453/453 [==============================] - 0s 107us/step - loss: 0.8559 - accuracy: 0.6534 - val_loss: 0.7620 - val_accuracy: 0.5590
Epoch 4/1000
453/453 [==============================] - 0s 108us/step - loss: 0.6859 - accuracy: 0.6909 - val_loss: 0.6040 - val_accuracy: 0.7077
Epoch 5/1000
453/453 [==============================] - 0s 116us/step - loss: 0.5397 - accuracy: 0.7483 - val_loss: 0.5447 - val_accuracy: 0.7487
Epoch 6/1000
453/453 [==============================] - 0s 252us/step - loss: 0.6278 - accuracy: 0.7550 - val_loss: 0.6452 - val_accuracy: 0.7487
Epoch 7/1000
453/453 [==============================] - 0s 316us/step - loss: 

Epoch 112/1000
453/453 [==============================] - 0s 201us/step - loss: 0.1966 - accuracy: 0.9117 - val_loss: 0.3495 - val_accuracy: 0.8513
Epoch 113/1000
453/453 [==============================] - 0s 299us/step - loss: 0.1940 - accuracy: 0.9029 - val_loss: 0.3549 - val_accuracy: 0.8513
Epoch 114/1000
453/453 [==============================] - 0s 141us/step - loss: 0.1958 - accuracy: 0.9183 - val_loss: 0.3459 - val_accuracy: 0.8769
Epoch 115/1000
453/453 [==============================] - 0s 115us/step - loss: 0.1901 - accuracy: 0.9249 - val_loss: 0.3661 - val_accuracy: 0.8615
Epoch 116/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2091 - accuracy: 0.9117 - val_loss: 0.3617 - val_accuracy: 0.8718
Epoch 117/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2025 - accuracy: 0.9161 - val_loss: 0.3702 - val_accuracy: 0.8718
Epoch 118/1000
453/453 [==============================] - 0s 163us/step - loss: 0.2403 - accuracy: 0.9073 - val_

453/453 [==============================] - 0s 101us/step - loss: 0.1590 - accuracy: 0.9360 - val_loss: 0.4102 - val_accuracy: 0.8872
Epoch 223/1000
453/453 [==============================] - 0s 117us/step - loss: 0.1621 - accuracy: 0.9316 - val_loss: 0.4187 - val_accuracy: 0.8923
Epoch 224/1000
453/453 [==============================] - 0s 126us/step - loss: 0.1620 - accuracy: 0.9183 - val_loss: 0.4184 - val_accuracy: 0.8821
Epoch 225/1000
453/453 [==============================] - 0s 133us/step - loss: 0.1781 - accuracy: 0.9249 - val_loss: 0.4141 - val_accuracy: 0.8821
Epoch 226/1000
453/453 [==============================] - 0s 116us/step - loss: 0.1740 - accuracy: 0.9227 - val_loss: 0.4145 - val_accuracy: 0.8718
Epoch 227/1000
453/453 [==============================] - 0s 105us/step - loss: 0.1746 - accuracy: 0.9227 - val_loss: 0.4347 - val_accuracy: 0.8667
Epoch 228/1000
453/453 [==============================] - 0s 108us/step - loss: 0.1820 - accuracy: 0.9227 - val_loss: 0.3927 - 

453/453 [==============================] - 0s 98us/step - loss: 0.1454 - accuracy: 0.9360 - val_loss: 0.4552 - val_accuracy: 0.9026
Epoch 333/1000
453/453 [==============================] - 0s 96us/step - loss: 0.1589 - accuracy: 0.9139 - val_loss: 0.4476 - val_accuracy: 0.8974
Epoch 334/1000
453/453 [==============================] - 0s 94us/step - loss: 0.1469 - accuracy: 0.9382 - val_loss: 0.4669 - val_accuracy: 0.8872
Epoch 335/1000
453/453 [==============================] - 0s 105us/step - loss: 0.1475 - accuracy: 0.9316 - val_loss: 0.4355 - val_accuracy: 0.9026
Epoch 336/1000
453/453 [==============================] - 0s 170us/step - loss: 0.1573 - accuracy: 0.9161 - val_loss: 0.4693 - val_accuracy: 0.8667
Epoch 337/1000
453/453 [==============================] - 0s 131us/step - loss: 0.1481 - accuracy: 0.9249 - val_loss: 0.4486 - val_accuracy: 0.8923
Epoch 338/1000
453/453 [==============================] - 0s 101us/step - loss: 0.1432 - accuracy: 0.9382 - val_loss: 0.4506 - val

453/453 [==============================] - 0s 97us/step - loss: 0.1344 - accuracy: 0.9382 - val_loss: 0.5058 - val_accuracy: 0.8974
Epoch 443/1000
453/453 [==============================] - 0s 242us/step - loss: 0.1357 - accuracy: 0.9316 - val_loss: 0.4961 - val_accuracy: 0.9026
Epoch 444/1000
453/453 [==============================] - 0s 290us/step - loss: 0.1476 - accuracy: 0.9249 - val_loss: 0.5162 - val_accuracy: 0.8923
Epoch 445/1000
453/453 [==============================] - 0s 216us/step - loss: 0.1343 - accuracy: 0.9382 - val_loss: 0.5201 - val_accuracy: 0.8872
Epoch 446/1000
453/453 [==============================] - 0s 258us/step - loss: 0.1326 - accuracy: 0.9360 - val_loss: 0.5022 - val_accuracy: 0.8974
Epoch 447/1000
453/453 [==============================] - 0s 239us/step - loss: 0.1312 - accuracy: 0.9404 - val_loss: 0.5254 - val_accuracy: 0.8923
Epoch 448/1000
453/453 [==============================] - 0s 114us/step - loss: 0.1392 - accuracy: 0.9316 - val_loss: 0.5088 - v

453/453 [==============================] - 0s 166us/step - loss: 0.3769 - accuracy: 0.9073 - val_loss: 0.5963 - val_accuracy: 0.8513
Epoch 553/1000
453/453 [==============================] - 0s 187us/step - loss: 0.1576 - accuracy: 0.9294 - val_loss: 0.5893 - val_accuracy: 0.8615
Epoch 554/1000
453/453 [==============================] - 0s 242us/step - loss: 0.1723 - accuracy: 0.9294 - val_loss: 0.5510 - val_accuracy: 0.8769
Epoch 555/1000
453/453 [==============================] - 0s 182us/step - loss: 0.1339 - accuracy: 0.9404 - val_loss: 0.5547 - val_accuracy: 0.8923
Epoch 556/1000
453/453 [==============================] - 0s 140us/step - loss: 0.1417 - accuracy: 0.9316 - val_loss: 0.5578 - val_accuracy: 0.8974
Epoch 557/1000
453/453 [==============================] - 0s 130us/step - loss: 0.1337 - accuracy: 0.9360 - val_loss: 0.5642 - val_accuracy: 0.8923
Epoch 558/1000
453/453 [==============================] - 0s 140us/step - loss: 0.1359 - accuracy: 0.9382 - val_loss: 0.5330 - 

Epoch 663/1000
453/453 [==============================] - 0s 102us/step - loss: 0.1304 - accuracy: 0.9404 - val_loss: 0.5846 - val_accuracy: 0.8974
Epoch 664/1000
453/453 [==============================] - 0s 92us/step - loss: 0.1288 - accuracy: 0.9404 - val_loss: 0.5899 - val_accuracy: 0.8769
Epoch 665/1000
453/453 [==============================] - 0s 100us/step - loss: 0.1340 - accuracy: 0.9272 - val_loss: 0.5841 - val_accuracy: 0.9026
Epoch 666/1000
453/453 [==============================] - 0s 97us/step - loss: 0.1262 - accuracy: 0.9426 - val_loss: 0.5896 - val_accuracy: 0.9026
Epoch 667/1000
453/453 [==============================] - 0s 95us/step - loss: 0.1330 - accuracy: 0.9360 - val_loss: 0.6235 - val_accuracy: 0.8974
Epoch 668/1000
453/453 [==============================] - 0s 102us/step - loss: 0.1290 - accuracy: 0.9338 - val_loss: 0.5849 - val_accuracy: 0.8974
Epoch 669/1000
453/453 [==============================] - 0s 101us/step - loss: 0.1312 - accuracy: 0.9382 - val_los

Epoch 774/1000
453/453 [==============================] - 0s 97us/step - loss: 0.1329 - accuracy: 0.9316 - val_loss: 0.6287 - val_accuracy: 0.9026
Epoch 775/1000
453/453 [==============================] - 0s 96us/step - loss: 0.1289 - accuracy: 0.9404 - val_loss: 0.6238 - val_accuracy: 0.8974
Epoch 776/1000
453/453 [==============================] - 0s 101us/step - loss: 0.1417 - accuracy: 0.9272 - val_loss: 0.6452 - val_accuracy: 0.9026
Epoch 777/1000
453/453 [==============================] - 0s 97us/step - loss: 0.1387 - accuracy: 0.9382 - val_loss: 0.6371 - val_accuracy: 0.9026
Epoch 778/1000
453/453 [==============================] - 0s 95us/step - loss: 0.1270 - accuracy: 0.9426 - val_loss: 0.6259 - val_accuracy: 0.9026
Epoch 779/1000
453/453 [==============================] - 0s 93us/step - loss: 0.1244 - accuracy: 0.9360 - val_loss: 0.6330 - val_accuracy: 0.8974
Epoch 780/1000
453/453 [==============================] - 0s 95us/step - loss: 0.1313 - accuracy: 0.9404 - val_loss: 

Epoch 885/1000
453/453 [==============================] - 0s 129us/step - loss: 0.1287 - accuracy: 0.9338 - val_loss: 0.6603 - val_accuracy: 0.8769
Epoch 886/1000
453/453 [==============================] - 0s 105us/step - loss: 0.1243 - accuracy: 0.9426 - val_loss: 0.6591 - val_accuracy: 0.9026
Epoch 887/1000
453/453 [==============================] - 0s 105us/step - loss: 0.1264 - accuracy: 0.9426 - val_loss: 0.6633 - val_accuracy: 0.9026
Epoch 888/1000
453/453 [==============================] - 0s 96us/step - loss: 0.1271 - accuracy: 0.9338 - val_loss: 0.6692 - val_accuracy: 0.9026
Epoch 889/1000
453/453 [==============================] - 0s 97us/step - loss: 0.1245 - accuracy: 0.9382 - val_loss: 0.6630 - val_accuracy: 0.9026
Epoch 890/1000
453/453 [==============================] - 0s 95us/step - loss: 0.1313 - accuracy: 0.9338 - val_loss: 0.6749 - val_accuracy: 0.9026
Epoch 891/1000
453/453 [==============================] - 0s 100us/step - loss: 0.1282 - accuracy: 0.9426 - val_los

Epoch 996/1000
453/453 [==============================] - 0s 96us/step - loss: 0.1275 - accuracy: 0.9316 - val_loss: 0.7002 - val_accuracy: 0.8974
Epoch 997/1000
453/453 [==============================] - 0s 97us/step - loss: 0.1264 - accuracy: 0.9272 - val_loss: 0.7055 - val_accuracy: 0.9077
Epoch 998/1000
453/453 [==============================] - 0s 96us/step - loss: 0.1278 - accuracy: 0.9426 - val_loss: 0.7418 - val_accuracy: 0.8974
Epoch 999/1000
453/453 [==============================] - 0s 96us/step - loss: 0.1345 - accuracy: 0.9161 - val_loss: 0.7125 - val_accuracy: 0.8974
Epoch 1000/1000
453/453 [==============================] - 0s 102us/step - loss: 0.1316 - accuracy: 0.9338 - val_loss: 0.7152 - val_accuracy: 0.9026


In [34]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

195/195 [==============================] - 0s 71us/step
over-sampling test accuracy: 90.77%


In [21]:
pred = model1_over.predict_classes(X_test_over)
pred

array([1, 0, 1, 1, 2, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 1, 2, 1, 1,
       2, 0, 1, 1, 0, 0, 0, 0, 2, 0, 2, 2, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 2, 0, 1, 2, 2, 2, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 2, 2, 1, 2, 2, 0, 0, 0, 1, 0, 1, 1, 2, 2, 0, 0, 0, 2,
       1, 2, 1, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 1, 0, 1, 1, 2, 1, 2,
       2, 1, 1, 1, 0, 2, 2, 2, 1, 0, 2, 1, 1, 2, 0, 0, 1, 0, 0, 2, 0, 2,
       0, 0, 0, 2, 2, 0, 2, 1, 0, 1, 2, 1, 0, 2, 2, 2, 2, 2, 2, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 2, 0, 2, 1, 2, 2, 2, 1, 1, 2, 0, 0, 2,
       1, 2, 2, 1, 2, 1, 0, 2, 2, 2, 2, 2, 1, 1, 2, 0, 1, 2, 1])

In [22]:
dat['pred'] = pred
dat

,0,test,pred
0,312,1,1
1,CFBRSa27,0,0
2,BCH-SA-01,1,1
3,GA27,1,1
4,NRS209,2,2
...,...,...,...
190,NRS209,2,2
191,NRS235,0,0
192,NRS240,1,1
193,NRS110,2,2


In [23]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,1.606453e-02,9.835255e-01,4.100458e-04
1,7.137315e-01,2.862685e-01,7.475717e-28
2,9.938727e-03,9.900613e-01,1.237838e-29
3,2.949264e-02,9.705073e-01,6.572662e-26
4,2.539522e-07,1.580527e-05,9.999839e-01
...,...,...,...
190,2.539522e-07,1.580527e-05,9.999839e-01
191,9.999998e-01,2.763784e-07,2.110893e-24
192,2.327191e-06,9.999976e-01,4.370972e-21
193,1.015389e-07,1.116668e-12,9.999999e-01


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [26]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p17sST.csv", index = False,
         header=None)

In [38]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 213us/step - loss: 0.1306 - accuracy: 0.9426 - val_loss: 0.7990 - val_accuracy: 0.9077
Epoch 2/1000
453/453 [==============================] - 0s 201us/step - loss: 0.1247 - accuracy: 0.9426 - val_loss: 0.7913 - val_accuracy: 0.9077
Epoch 3/1000
453/453 [==============================] - 0s 187us/step - loss: 0.1195 - accuracy: 0.9426 - val_loss: 0.7858 - val_accuracy: 0.8872
Epoch 4/1000
453/453 [==============================] - 0s 179us/step - loss: 0.1199 - accuracy: 0.9426 - val_loss: 0.7794 - val_accuracy: 0.9077
Epoch 5/1000
453/453 [==============================] - 0s 176us/step - loss: 0.1168 - accuracy: 0.9426 - val_loss: 0.7960 - val_accuracy: 0.9077
Epoch 6/1000
453/453 [==============================] - 0s 177us/step - loss: 0.1248 - accuracy: 0.9404 - val_loss: 0.7791 - val_accuracy: 0.8821
Epoch 7/1000
453/453 [==============================] - 0s 182us/step - loss: 

Epoch 112/1000
453/453 [==============================] - 0s 276us/step - loss: 0.1177 - accuracy: 0.9448 - val_loss: 0.7750 - val_accuracy: 0.9077
Epoch 113/1000
453/453 [==============================] - 0s 269us/step - loss: 0.1151 - accuracy: 0.9426 - val_loss: 0.7750 - val_accuracy: 0.9077
Epoch 114/1000
453/453 [==============================] - 0s 232us/step - loss: 0.1188 - accuracy: 0.9360 - val_loss: 0.7789 - val_accuracy: 0.9077
Epoch 115/1000
453/453 [==============================] - 0s 164us/step - loss: 0.1185 - accuracy: 0.9404 - val_loss: 0.7562 - val_accuracy: 0.9077
Epoch 116/1000
453/453 [==============================] - 0s 171us/step - loss: 0.1209 - accuracy: 0.9448 - val_loss: 0.7746 - val_accuracy: 0.9077
Epoch 117/1000
453/453 [==============================] - 0s 173us/step - loss: 0.1193 - accuracy: 0.9360 - val_loss: 0.7702 - val_accuracy: 0.9077
Epoch 118/1000
453/453 [==============================] - 0s 172us/step - loss: 0.1166 - accuracy: 0.9448 - val_

Epoch 222/1000
453/453 [==============================] - 0s 148us/step - loss: 0.1153 - accuracy: 0.9448 - val_loss: 0.7485 - val_accuracy: 0.9077
Epoch 223/1000
453/453 [==============================] - 0s 123us/step - loss: 0.1169 - accuracy: 0.9448 - val_loss: 0.7707 - val_accuracy: 0.9077
Epoch 224/1000
453/453 [==============================] - 0s 268us/step - loss: 0.1171 - accuracy: 0.9338 - val_loss: 0.7579 - val_accuracy: 0.8872
Epoch 225/1000
453/453 [==============================] - 0s 301us/step - loss: 0.1145 - accuracy: 0.9448 - val_loss: 0.7779 - val_accuracy: 0.9077
Epoch 226/1000
453/453 [==============================] - 0s 251us/step - loss: 0.1213 - accuracy: 0.9404 - val_loss: 0.7906 - val_accuracy: 0.9077
Epoch 227/1000
453/453 [==============================] - 0s 226us/step - loss: 0.1230 - accuracy: 0.9360 - val_loss: 0.7480 - val_accuracy: 0.9077
Epoch 228/1000
453/453 [==============================] - 0s 401us/step - loss: 0.1171 - accuracy: 0.9249 - val_

453/453 [==============================] - 0s 110us/step - loss: 0.1118 - accuracy: 0.9404 - val_loss: 0.7278 - val_accuracy: 0.8872
Epoch 333/1000
453/453 [==============================] - 0s 114us/step - loss: 0.1170 - accuracy: 0.9360 - val_loss: 0.7879 - val_accuracy: 0.8872
Epoch 334/1000
453/453 [==============================] - 0s 107us/step - loss: 0.1190 - accuracy: 0.9426 - val_loss: 0.7452 - val_accuracy: 0.9026
Epoch 335/1000
453/453 [==============================] - 0s 102us/step - loss: 0.1214 - accuracy: 0.9404 - val_loss: 0.7239 - val_accuracy: 0.9026
Epoch 336/1000
453/453 [==============================] - 0s 109us/step - loss: 0.1130 - accuracy: 0.9382 - val_loss: 0.7263 - val_accuracy: 0.9077
Epoch 337/1000
453/453 [==============================] - 0s 113us/step - loss: 0.1282 - accuracy: 0.9382 - val_loss: 0.7444 - val_accuracy: 0.9077
Epoch 338/1000
453/453 [==============================] - 0s 133us/step - loss: 0.1232 - accuracy: 0.9426 - val_loss: 0.7654 - 

453/453 [==============================] - 0s 107us/step - loss: 0.1132 - accuracy: 0.9448 - val_loss: 0.7876 - val_accuracy: 0.9077
Epoch 443/1000
453/453 [==============================] - 0s 98us/step - loss: 0.1131 - accuracy: 0.9404 - val_loss: 0.7759 - val_accuracy: 0.9077
Epoch 444/1000
453/453 [==============================] - 0s 94us/step - loss: 0.1151 - accuracy: 0.9448 - val_loss: 0.7867 - val_accuracy: 0.9077
Epoch 445/1000
453/453 [==============================] - 0s 99us/step - loss: 0.1195 - accuracy: 0.9360 - val_loss: 0.7886 - val_accuracy: 0.9077
Epoch 446/1000
453/453 [==============================] - 0s 103us/step - loss: 0.1152 - accuracy: 0.9448 - val_loss: 0.8089 - val_accuracy: 0.9077
Epoch 447/1000
453/453 [==============================] - 0s 134us/step - loss: 0.1151 - accuracy: 0.9426 - val_loss: 0.7939 - val_accuracy: 0.9077
Epoch 448/1000
453/453 [==============================] - 0s 116us/step - loss: 0.1240 - accuracy: 0.9360 - val_loss: 0.7908 - val

453/453 [==============================] - 0s 105us/step - loss: 0.1188 - accuracy: 0.9426 - val_loss: 0.8957 - val_accuracy: 0.9077
Epoch 553/1000
453/453 [==============================] - 0s 98us/step - loss: 0.1145 - accuracy: 0.9382 - val_loss: 0.8779 - val_accuracy: 0.9077
Epoch 554/1000
453/453 [==============================] - 0s 107us/step - loss: 0.1141 - accuracy: 0.9426 - val_loss: 0.8622 - val_accuracy: 0.9077
Epoch 555/1000
453/453 [==============================] - 0s 100us/step - loss: 0.1202 - accuracy: 0.9404 - val_loss: 0.8803 - val_accuracy: 0.9077
Epoch 556/1000
453/453 [==============================] - 0s 99us/step - loss: 0.1170 - accuracy: 0.9404 - val_loss: 0.8842 - val_accuracy: 0.9077
Epoch 557/1000
453/453 [==============================] - 0s 101us/step - loss: 0.1117 - accuracy: 0.9382 - val_loss: 0.8585 - val_accuracy: 0.8821
Epoch 558/1000
453/453 [==============================] - 0s 100us/step - loss: 0.1163 - accuracy: 0.9404 - val_loss: 0.8780 - va

Epoch 663/1000
453/453 [==============================] - 0s 100us/step - loss: 0.1140 - accuracy: 0.9404 - val_loss: 0.9352 - val_accuracy: 0.8974
Epoch 664/1000
453/453 [==============================] - 0s 96us/step - loss: 0.1182 - accuracy: 0.9360 - val_loss: 0.9127 - val_accuracy: 0.9077
Epoch 665/1000
453/453 [==============================] - 0s 104us/step - loss: 0.1150 - accuracy: 0.9404 - val_loss: 0.9081 - val_accuracy: 0.9077
Epoch 666/1000
453/453 [==============================] - 0s 107us/step - loss: 0.1126 - accuracy: 0.9448 - val_loss: 0.9117 - val_accuracy: 0.9077
Epoch 667/1000
453/453 [==============================] - 0s 105us/step - loss: 0.1161 - accuracy: 0.9448 - val_loss: 0.9093 - val_accuracy: 0.9077
Epoch 668/1000
453/453 [==============================] - 0s 106us/step - loss: 0.1164 - accuracy: 0.9426 - val_loss: 0.9127 - val_accuracy: 0.9077
Epoch 669/1000
453/453 [==============================] - 0s 105us/step - loss: 0.1140 - accuracy: 0.9382 - val_l

453/453 [==============================] - 0s 99us/step - loss: 0.1157 - accuracy: 0.9448 - val_loss: 0.9748 - val_accuracy: 0.9077
Epoch 774/1000
453/453 [==============================] - 0s 100us/step - loss: 0.1213 - accuracy: 0.9227 - val_loss: 0.9533 - val_accuracy: 0.8821
Epoch 775/1000
453/453 [==============================] - 0s 96us/step - loss: 0.1171 - accuracy: 0.9360 - val_loss: 0.9599 - val_accuracy: 0.9077
Epoch 776/1000
453/453 [==============================] - 0s 97us/step - loss: 0.1134 - accuracy: 0.9448 - val_loss: 0.9435 - val_accuracy: 0.9077
Epoch 777/1000
453/453 [==============================] - 0s 105us/step - loss: 0.1153 - accuracy: 0.9448 - val_loss: 0.9481 - val_accuracy: 0.9077
Epoch 778/1000
453/453 [==============================] - 0s 106us/step - loss: 0.1137 - accuracy: 0.9448 - val_loss: 0.9666 - val_accuracy: 0.9077
Epoch 779/1000
453/453 [==============================] - 0s 121us/step - loss: 0.1145 - accuracy: 0.9404 - val_loss: 0.9467 - val

453/453 [==============================] - 0s 99us/step - loss: 0.1141 - accuracy: 0.9448 - val_loss: 0.8949 - val_accuracy: 0.9077
Epoch 884/1000
453/453 [==============================] - 0s 98us/step - loss: 0.1136 - accuracy: 0.9382 - val_loss: 0.9361 - val_accuracy: 0.9077
Epoch 885/1000
453/453 [==============================] - 0s 99us/step - loss: 0.1171 - accuracy: 0.9404 - val_loss: 0.9687 - val_accuracy: 0.8974
Epoch 886/1000
453/453 [==============================] - 0s 98us/step - loss: 0.1186 - accuracy: 0.9205 - val_loss: 0.9314 - val_accuracy: 0.9077
Epoch 887/1000
453/453 [==============================] - 0s 105us/step - loss: 0.1118 - accuracy: 0.9448 - val_loss: 0.9281 - val_accuracy: 0.9077
Epoch 888/1000
453/453 [==============================] - 0s 106us/step - loss: 0.1170 - accuracy: 0.9404 - val_loss: 0.9555 - val_accuracy: 0.9077
Epoch 889/1000
453/453 [==============================] - 0s 99us/step - loss: 0.1159 - accuracy: 0.9382 - val_loss: 0.9466 - val_a

453/453 [==============================] - 0s 97us/step - loss: 0.1129 - accuracy: 0.9448 - val_loss: 0.9751 - val_accuracy: 0.9077
Epoch 994/1000
453/453 [==============================] - 0s 105us/step - loss: 0.1161 - accuracy: 0.9448 - val_loss: 0.9806 - val_accuracy: 0.9077
Epoch 995/1000
453/453 [==============================] - 0s 106us/step - loss: 0.1217 - accuracy: 0.9382 - val_loss: 0.9447 - val_accuracy: 0.9077
Epoch 996/1000
453/453 [==============================] - 0s 107us/step - loss: 0.1163 - accuracy: 0.9448 - val_loss: 0.9906 - val_accuracy: 0.9077
Epoch 997/1000
453/453 [==============================] - 0s 99us/step - loss: 0.1144 - accuracy: 0.9448 - val_loss: 0.9740 - val_accuracy: 0.9077
Epoch 998/1000
453/453 [==============================] - 0s 99us/step - loss: 0.1115 - accuracy: 0.9448 - val_loss: 1.0029 - val_accuracy: 0.8974
Epoch 999/1000
453/453 [==============================] - 0s 98us/step - loss: 0.1215 - accuracy: 0.9294 - val_loss: 0.8695 - val_

In [39]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 93.90%


In [18]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [19]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [20]:
y_prob = df_proba[df_proba['phage']=='p0017SkpresabsSTCC_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[1.60645320e-02, 9.83525500e-01, 4.10045820e-04],
       [7.13731500e-01, 2.86268500e-01, 7.47571700e-28],
       [9.93872700e-03, 9.90061300e-01, 1.23783810e-29],
       [2.94926450e-02, 9.70507300e-01, 6.57266200e-26],
       [2.53952150e-07, 1.58052700e-05, 9.99983900e-01],
       [1.60645320e-02, 9.83525500e-01, 4.10045820e-04],
       [1.62753500e-18, 1.00000000e+00, 1.00791256e-22],
       [9.09360800e-01, 8.95153600e-02, 1.12385230e-03],
       [7.13731500e-01, 2.86268500e-01, 7.47571700e-28],
       [1.00000000e+00, 6.80996600e-10, 2.92078750e-12],
       [1.86638670e-15, 1.00000000e+00, 0.00000000e+00],
       [1.09199835e-14, 1.00000000e+00, 2.06061190e-12],
       [7.13731500e-01, 2.86268500e-01, 7.47571700e-28],
       [1.00000000e+00, 1.96283400e-14, 2.47762600e-19],
       [7.13731500e-01, 2.86268500e-01, 7.47571700e-28],
       [7.13731500e-01, 2.86268500e-01, 7.47571700e-28],
       [2.53952150e-07, 1.58052700e-05, 9.99983900e-01],
       [7.77183700e-01, 2.22816

In [21]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [22]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.9517159763313608

In [23]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [24]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.9517159763313608

In [25]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [26]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [27]:
dat2

,0,test
0,NRS110,2
1,NRS254,1
2,BCH-SA-09,0
3,NRS177,0
4,GA27,1
...,...,...
190,NRS001,1
191,NRS209,2
192,NRS272,0
193,NRS110,2


In [28]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [29]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [32]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 282us/step - loss: 10.1512 - accuracy: 0.3091 - val_loss: 4.0817 - val_accuracy: 0.3333
Epoch 2/1000
453/453 [==============================] - 0s 74us/step - loss: 3.2749 - accuracy: 0.4437 - val_loss: 1.5090 - val_accuracy: 0.5077
Epoch 3/1000
453/453 [==============================] - 0s 88us/step - loss: 1.2649 - accuracy: 0.5982 - val_loss: 0.9424 - val_accuracy: 0.5333
Epoch 4/1000
453/453 [==============================] - 0s 80us/step - loss: 0.9833 - accuracy: 0.5828 - val_loss: 0.6985 - val_accuracy: 0.6974
Epoch 5/1000
453/453 [==============================] - 0s 74us/step - loss: 0.7292 - accuracy: 0.6623 - val_loss: 0.6565 - val_accuracy: 0.6923
Epoch 6/1000
453/453 [==============================] - 0s 84us/step - loss: 0.6180 - accuracy: 0.7108 - val_loss: 0.5906 - val_accuracy: 0.6872
Epoch 7/1000
453/453 [==============================] - 0s 82us/step - loss: 0.566

Epoch 112/1000
453/453 [==============================] - 0s 124us/step - loss: 0.2323 - accuracy: 0.9117 - val_loss: 0.4126 - val_accuracy: 0.8923
Epoch 113/1000
453/453 [==============================] - 0s 77us/step - loss: 0.2253 - accuracy: 0.9117 - val_loss: 0.4298 - val_accuracy: 0.8923
Epoch 114/1000
453/453 [==============================] - 0s 79us/step - loss: 0.2282 - accuracy: 0.9095 - val_loss: 0.4096 - val_accuracy: 0.9026
Epoch 115/1000
453/453 [==============================] - 0s 172us/step - loss: 0.2292 - accuracy: 0.9095 - val_loss: 0.4289 - val_accuracy: 0.8923
Epoch 116/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2260 - accuracy: 0.9117 - val_loss: 0.4304 - val_accuracy: 0.8923
Epoch 117/1000
453/453 [==============================] - 0s 86us/step - loss: 0.2336 - accuracy: 0.9051 - val_loss: 0.4139 - val_accuracy: 0.9026
Epoch 118/1000
453/453 [==============================] - 0s 66us/step - loss: 0.2278 - accuracy: 0.9139 - val_loss

Epoch 168/1000
453/453 [==============================] - 0s 58us/step - loss: 0.2018 - accuracy: 0.9183 - val_loss: 0.4641 - val_accuracy: 0.8923
Epoch 169/1000
453/453 [==============================] - 0s 61us/step - loss: 0.2051 - accuracy: 0.9161 - val_loss: 0.5351 - val_accuracy: 0.8769
Epoch 170/1000
453/453 [==============================] - 0s 55us/step - loss: 0.2591 - accuracy: 0.8852 - val_loss: 0.6565 - val_accuracy: 0.8821
Epoch 171/1000
453/453 [==============================] - 0s 57us/step - loss: 0.3128 - accuracy: 0.9007 - val_loss: 0.4777 - val_accuracy: 0.8923
Epoch 172/1000
453/453 [==============================] - 0s 56us/step - loss: 0.2196 - accuracy: 0.9007 - val_loss: 0.4635 - val_accuracy: 0.9026
Epoch 173/1000
453/453 [==============================] - 0s 56us/step - loss: 0.2212 - accuracy: 0.9095 - val_loss: 0.4896 - val_accuracy: 0.8821
Epoch 174/1000
453/453 [==============================] - 0s 55us/step - loss: 0.3347 - accuracy: 0.8940 - val_loss: 0

Epoch 224/1000
453/453 [==============================] - 0s 96us/step - loss: 0.1857 - accuracy: 0.9227 - val_loss: 0.5252 - val_accuracy: 0.8923
Epoch 225/1000
453/453 [==============================] - 0s 106us/step - loss: 0.1813 - accuracy: 0.9227 - val_loss: 0.5346 - val_accuracy: 0.9026
Epoch 226/1000
453/453 [==============================] - 0s 90us/step - loss: 0.1879 - accuracy: 0.9272 - val_loss: 0.5405 - val_accuracy: 0.9026
Epoch 227/1000
453/453 [==============================] - 0s 103us/step - loss: 0.1871 - accuracy: 0.9227 - val_loss: 0.5212 - val_accuracy: 0.9128
Epoch 228/1000
453/453 [==============================] - 0s 73us/step - loss: 0.1798 - accuracy: 0.9294 - val_loss: 0.5433 - val_accuracy: 0.9026
Epoch 229/1000
453/453 [==============================] - 0s 68us/step - loss: 0.1829 - accuracy: 0.9227 - val_loss: 0.5267 - val_accuracy: 0.9026
Epoch 230/1000
453/453 [==============================] - 0s 65us/step - loss: 0.1790 - accuracy: 0.9272 - val_loss:

Epoch 335/1000
453/453 [==============================] - 0s 85us/step - loss: 0.2859 - accuracy: 0.9007 - val_loss: 0.6196 - val_accuracy: 0.9077
Epoch 336/1000
453/453 [==============================] - 0s 94us/step - loss: 0.2566 - accuracy: 0.9073 - val_loss: 0.7471 - val_accuracy: 0.8923
Epoch 337/1000
453/453 [==============================] - 0s 85us/step - loss: 0.1966 - accuracy: 0.9227 - val_loss: 0.6260 - val_accuracy: 0.9077
Epoch 338/1000
453/453 [==============================] - 0s 81us/step - loss: 0.5005 - accuracy: 0.8455 - val_loss: 0.9543 - val_accuracy: 0.8718
Epoch 339/1000
453/453 [==============================] - 0s 90us/step - loss: 0.2668 - accuracy: 0.9117 - val_loss: 0.8619 - val_accuracy: 0.8718
Epoch 340/1000
453/453 [==============================] - 0s 100us/step - loss: 0.4596 - accuracy: 0.8985 - val_loss: 1.1914 - val_accuracy: 0.8410
Epoch 341/1000
453/453 [==============================] - 0s 91us/step - loss: 0.3431 - accuracy: 0.8852 - val_loss: 

...

Epoch 727/1000
453/453 [==============================] - 0s 72us/step - loss: 0.7769 - accuracy: 0.8852 - val_loss: 0.9199 - val_accuracy: 0.8923
Epoch 728/1000
453/453 [==============================] - 0s 67us/step - loss: 0.2736 - accuracy: 0.9183 - val_loss: 0.8674 - val_accuracy: 0.9026
Epoch 729/1000
453/453 [==============================] - 0s 66us/step - loss: 0.2609 - accuracy: 0.9205 - val_loss: 0.8569 - val_accuracy: 0.9231
Epoch 730/1000
453/453 [==============================] - 0s 66us/step - loss: 0.2331 - accuracy: 0.9161 - val_loss: 1.4017 - val_accuracy: 0.8615
Epoch 731/1000
453/453 [==============================] - 0s 71us/step - loss: 0.2100 - accuracy: 0.9227 - val_loss: 1.2107 - val_accuracy: 0.8821
Epoch 732/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1510 - accuracy: 0.9316 - val_loss: 0.8625 - val_accuracy: 0.9128
Epoch 733/1000
453/453 [==============================] - 0s 65us/step - loss: 0.1376 - accuracy: 0.9316 - val_loss: 0

Epoch 783/1000
453/453 [==============================] - 0s 66us/step - loss: 0.1364 - accuracy: 0.9382 - val_loss: 0.9209 - val_accuracy: 0.9026
Epoch 784/1000
453/453 [==============================] - 0s 72us/step - loss: 0.1359 - accuracy: 0.9382 - val_loss: 0.9261 - val_accuracy: 0.8872
Epoch 785/1000
453/453 [==============================] - 0s 75us/step - loss: 0.1354 - accuracy: 0.9316 - val_loss: 0.9257 - val_accuracy: 0.9026
Epoch 786/1000
453/453 [==============================] - 0s 72us/step - loss: 0.1360 - accuracy: 0.9382 - val_loss: 0.9216 - val_accuracy: 0.9026
Epoch 787/1000
453/453 [==============================] - 0s 69us/step - loss: 0.1365 - accuracy: 0.9338 - val_loss: 0.9064 - val_accuracy: 0.9026
Epoch 788/1000
453/453 [==============================] - 0s 68us/step - loss: 0.1396 - accuracy: 0.9316 - val_loss: 0.9148 - val_accuracy: 0.9077
Epoch 789/1000
453/453 [==============================] - 0s 68us/step - loss: 0.1358 - accuracy: 0.9360 - val_loss: 0

Epoch 839/1000
453/453 [==============================] - 0s 70us/step - loss: 0.1362 - accuracy: 0.9316 - val_loss: 0.9356 - val_accuracy: 0.9128
Epoch 840/1000
453/453 [==============================] - 0s 73us/step - loss: 0.1326 - accuracy: 0.9382 - val_loss: 0.9405 - val_accuracy: 0.9128
Epoch 841/1000
453/453 [==============================] - 0s 68us/step - loss: 0.1307 - accuracy: 0.9382 - val_loss: 0.9469 - val_accuracy: 0.9128
Epoch 842/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1334 - accuracy: 0.9338 - val_loss: 0.9365 - val_accuracy: 0.9128
Epoch 843/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1425 - accuracy: 0.9382 - val_loss: 0.9682 - val_accuracy: 0.8872
Epoch 844/1000
453/453 [==============================] - 0s 65us/step - loss: 0.1352 - accuracy: 0.9360 - val_loss: 0.9421 - val_accuracy: 0.9128
Epoch 845/1000
453/453 [==============================] - 0s 72us/step - loss: 0.1379 - accuracy: 0.9338 - val_loss: 0

Epoch 895/1000
453/453 [==============================] - 0s 72us/step - loss: 0.1441 - accuracy: 0.9360 - val_loss: 0.9985 - val_accuracy: 0.8923
Epoch 896/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1323 - accuracy: 0.9382 - val_loss: 1.0024 - val_accuracy: 0.9026
Epoch 897/1000
453/453 [==============================] - 0s 65us/step - loss: 0.1311 - accuracy: 0.9382 - val_loss: 1.0289 - val_accuracy: 0.8974
Epoch 898/1000
453/453 [==============================] - 0s 73us/step - loss: 0.1371 - accuracy: 0.9316 - val_loss: 0.9989 - val_accuracy: 0.9077
Epoch 899/1000
453/453 [==============================] - 0s 66us/step - loss: 0.1310 - accuracy: 0.9382 - val_loss: 0.9900 - val_accuracy: 0.9128
Epoch 900/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1319 - accuracy: 0.9316 - val_loss: 1.0079 - val_accuracy: 0.9026
Epoch 901/1000
453/453 [==============================] - 0s 69us/step - loss: 0.1461 - accuracy: 0.9272 - val_loss: 1

Epoch 951/1000
453/453 [==============================] - 0s 68us/step - loss: 0.1349 - accuracy: 0.9382 - val_loss: 1.0267 - val_accuracy: 0.9128
Epoch 952/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1361 - accuracy: 0.9360 - val_loss: 1.0190 - val_accuracy: 0.9128
Epoch 953/1000
453/453 [==============================] - 0s 74us/step - loss: 0.1327 - accuracy: 0.9382 - val_loss: 1.0363 - val_accuracy: 0.8923
Epoch 954/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1349 - accuracy: 0.9404 - val_loss: 1.0323 - val_accuracy: 0.9077
Epoch 955/1000
453/453 [==============================] - 0s 72us/step - loss: 0.1342 - accuracy: 0.9382 - val_loss: 1.0338 - val_accuracy: 0.9077
Epoch 956/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1315 - accuracy: 0.9316 - val_loss: 1.0207 - val_accuracy: 0.9128
Epoch 957/1000
453/453 [==============================] - 0s 75us/step - loss: 0.1335 - accuracy: 0.9316 - val_loss: 1

In [51]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

195/195 [==============================] - 0s 63us/step
over-sampling test accuracy: 90.77%


In [34]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([2, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 0, 2, 2, 1, 2, 0, 1, 1, 0, 0, 0,
       2, 1, 1, 1, 1, 1, 0, 2, 0, 2, 1, 2, 1, 1, 2, 0, 1, 2, 1, 2, 2, 0,
       2, 0, 0, 2, 2, 0, 1, 1, 1, 2, 1, 0, 1, 0, 2, 2, 2, 2, 0, 1, 0, 2,
       0, 0, 2, 0, 1, 0, 1, 0, 0, 1, 2, 0, 1, 1, 1, 2, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 2, 0, 2, 1, 0, 0, 1, 1, 2, 2, 0, 2, 0, 1, 1, 2, 1, 1,
       2, 2, 1, 2, 2, 1, 0, 0, 2, 0, 1, 1, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2,
       0, 0, 1, 2, 2, 0, 1, 0, 2, 1, 0, 0, 2, 1, 2, 0, 0, 2, 1, 2, 1, 1,
       1, 2, 1, 2, 0, 1, 0, 1, 2, 0, 2, 1, 2, 0, 0, 2, 1, 2, 2, 1, 1, 2,
       1, 2, 1, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 1, 2, 0, 2, 0])

In [35]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,NRS110,2,2
1,NRS254,1,1
2,BCH-SA-09,0,1
3,NRS177,0,1
4,GA27,1,1
...,...,...,...
190,NRS001,1,1
191,NRS209,2,2
192,NRS272,0,0
193,NRS110,2,2


In [36]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [37]:
dat_proba2

,0,1,2
0,7.079149e-07,1.541663e-07,9.999992e-01
1,1.146303e-09,9.999999e-01,9.944006e-08
2,2.094643e-01,7.905357e-01,9.975265e-10
3,1.580161e-03,9.983804e-01,3.942445e-05
4,4.175024e-02,9.582497e-01,2.645318e-11
...,...,...,...
190,1.626247e-01,8.373753e-01,4.655971e-11
191,4.441976e-06,9.701334e-06,9.999858e-01
192,5.212452e-01,4.668341e-01,1.192073e-02
193,7.079149e-07,1.541663e-07,9.999992e-01


In [38]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [39]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p17sST.csv", index = False,
         header=None)

In [55]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 112us/step - loss: 0.1368 - accuracy: 0.9360 - val_loss: 0.5780 - val_accuracy: 0.9077
Epoch 2/1000
453/453 [==============================] - 0s 85us/step - loss: 0.1400 - accuracy: 0.9338 - val_loss: 0.5788 - val_accuracy: 0.9077
Epoch 3/1000
453/453 [==============================] - 0s 90us/step - loss: 0.1428 - accuracy: 0.9382 - val_loss: 0.6183 - val_accuracy: 0.9077
Epoch 4/1000
453/453 [==============================] - 0s 72us/step - loss: 0.1582 - accuracy: 0.9007 - val_loss: 0.6277 - val_accuracy: 0.9128
Epoch 5/1000
453/453 [==============================] - 0s 68us/step - loss: 0.1424 - accuracy: 0.9382 - val_loss: 0.6204 - val_accuracy: 0.9077
Epoch 6/1000
453/453 [==============================] - 0s 66us/step - loss: 0.1385 - accuracy: 0.9382 - val_loss: 0.6438 - val_accuracy: 0.9077
Epoch 7/1000
453/453 [==============================] - 0s 66us/step - loss: 0.1387

Epoch 57/1000
453/453 [==============================] - 0s 91us/step - loss: 0.1387 - accuracy: 0.9382 - val_loss: 0.6119 - val_accuracy: 0.9128
Epoch 58/1000
453/453 [==============================] - 0s 94us/step - loss: 0.1469 - accuracy: 0.9338 - val_loss: 0.6425 - val_accuracy: 0.9128
Epoch 59/1000
453/453 [==============================] - 0s 413us/step - loss: 0.1404 - accuracy: 0.9360 - val_loss: 0.6460 - val_accuracy: 0.9128
Epoch 60/1000
453/453 [==============================] - 0s 128us/step - loss: 0.1353 - accuracy: 0.9382 - val_loss: 0.6854 - val_accuracy: 0.9128
Epoch 61/1000
453/453 [==============================] - 0s 78us/step - loss: 0.1389 - accuracy: 0.9382 - val_loss: 0.7614 - val_accuracy: 0.9026
Epoch 62/1000
453/453 [==============================] - 0s 135us/step - loss: 0.1440 - accuracy: 0.9073 - val_loss: 0.6417 - val_accuracy: 0.9128
Epoch 63/1000
453/453 [==============================] - 0s 132us/step - loss: 0.1392 - accuracy: 0.9382 - val_loss: 0.68

Epoch 113/1000
453/453 [==============================] - 0s 122us/step - loss: 0.1399 - accuracy: 0.9382 - val_loss: 0.6286 - val_accuracy: 0.9128
Epoch 114/1000
453/453 [==============================] - 0s 111us/step - loss: 0.1404 - accuracy: 0.9360 - val_loss: 0.6910 - val_accuracy: 0.9128
Epoch 115/1000
453/453 [==============================] - 0s 107us/step - loss: 0.1482 - accuracy: 0.9338 - val_loss: 0.7453 - val_accuracy: 0.9026
Epoch 116/1000
453/453 [==============================] - 0s 93us/step - loss: 0.1368 - accuracy: 0.9360 - val_loss: 0.7672 - val_accuracy: 0.9026
Epoch 117/1000
453/453 [==============================] - 0s 87us/step - loss: 0.1350 - accuracy: 0.9382 - val_loss: 0.6649 - val_accuracy: 0.9128
Epoch 118/1000
453/453 [==============================] - 0s 96us/step - loss: 0.1381 - accuracy: 0.9382 - val_loss: 0.6779 - val_accuracy: 0.9128
Epoch 119/1000
453/453 [==============================] - 0s 85us/step - loss: 0.1384 - accuracy: 0.9382 - val_loss

453/453 [==============================] - 0s 130us/step - loss: 0.1499 - accuracy: 0.9382 - val_loss: 0.6938 - val_accuracy: 0.9128
Epoch 224/1000
453/453 [==============================] - 0s 123us/step - loss: 0.1410 - accuracy: 0.9338 - val_loss: 0.6448 - val_accuracy: 0.9128
Epoch 225/1000
453/453 [==============================] - 0s 131us/step - loss: 0.1381 - accuracy: 0.9382 - val_loss: 0.6731 - val_accuracy: 0.9128
Epoch 226/1000
453/453 [==============================] - 0s 120us/step - loss: 0.1340 - accuracy: 0.9382 - val_loss: 0.6854 - val_accuracy: 0.9128
Epoch 227/1000
453/453 [==============================] - 0s 114us/step - loss: 0.1392 - accuracy: 0.9338 - val_loss: 0.6707 - val_accuracy: 0.9128
Epoch 228/1000
453/453 [==============================] - 0s 123us/step - loss: 0.1340 - accuracy: 0.9382 - val_loss: 0.6679 - val_accuracy: 0.9128
Epoch 229/1000
453/453 [==============================] - 0s 149us/step - loss: 0.1320 - accuracy: 0.9382 - val_loss: 0.6727 - 

Epoch 279/1000
453/453 [==============================] - 0s 113us/step - loss: 0.1347 - accuracy: 0.9382 - val_loss: 0.6765 - val_accuracy: 0.9077
Epoch 280/1000
453/453 [==============================] - 0s 75us/step - loss: 0.1323 - accuracy: 0.9360 - val_loss: 0.6882 - val_accuracy: 0.9128
Epoch 281/1000
453/453 [==============================] - 0s 87us/step - loss: 0.1360 - accuracy: 0.9382 - val_loss: 0.7115 - val_accuracy: 0.9077
Epoch 282/1000
453/453 [==============================] - 0s 79us/step - loss: 0.1349 - accuracy: 0.9360 - val_loss: 0.6804 - val_accuracy: 0.9128
Epoch 283/1000
453/453 [==============================] - 0s 81us/step - loss: 0.1338 - accuracy: 0.9382 - val_loss: 0.6802 - val_accuracy: 0.9077
Epoch 284/1000
453/453 [==============================] - 0s 119us/step - loss: 0.1401 - accuracy: 0.9360 - val_loss: 0.6985 - val_accuracy: 0.9077
Epoch 285/1000
453/453 [==============================] - 0s 82us/step - loss: 0.1339 - accuracy: 0.9382 - val_loss:

...

Epoch 613/1000
453/453 [==============================] - 0s 96us/step - loss: 0.1364 - accuracy: 0.9360 - val_loss: 0.8162 - val_accuracy: 0.9128
Epoch 614/1000
453/453 [==============================] - 0s 88us/step - loss: 0.1355 - accuracy: 0.9382 - val_loss: 0.8406 - val_accuracy: 0.9128
Epoch 615/1000
453/453 [==============================] - 0s 75us/step - loss: 0.1325 - accuracy: 0.9316 - val_loss: 0.7357 - val_accuracy: 0.9128
Epoch 616/1000
453/453 [==============================] - 0s 65us/step - loss: 0.1311 - accuracy: 0.9382 - val_loss: 0.7058 - val_accuracy: 0.9128
Epoch 617/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1387 - accuracy: 0.9382 - val_loss: 0.7735 - val_accuracy: 0.9128
Epoch 618/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1367 - accuracy: 0.9382 - val_loss: 0.7055 - val_accuracy: 0.9128
Epoch 619/1000
453/453 [==============================] - 0s 62us/step - loss: 0.1326 - accuracy: 0.9382 - val_loss: 0

Epoch 669/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1302 - accuracy: 0.9382 - val_loss: 0.7671 - val_accuracy: 0.9128
Epoch 670/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1378 - accuracy: 0.9382 - val_loss: 0.7850 - val_accuracy: 0.9128
Epoch 671/1000
453/453 [==============================] - 0s 75us/step - loss: 0.1534 - accuracy: 0.8918 - val_loss: 0.8716 - val_accuracy: 0.9128
Epoch 672/1000
453/453 [==============================] - 0s 62us/step - loss: 0.1350 - accuracy: 0.9316 - val_loss: 0.7933 - val_accuracy: 0.9128
Epoch 673/1000
453/453 [==============================] - 0s 73us/step - loss: 0.1418 - accuracy: 0.9382 - val_loss: 0.8157 - val_accuracy: 0.9128
Epoch 674/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1453 - accuracy: 0.9338 - val_loss: 0.8305 - val_accuracy: 0.9128
Epoch 675/1000
453/453 [==============================] - 0s 73us/step - loss: 0.1351 - accuracy: 0.9360 - val_loss: 0

Epoch 780/1000
453/453 [==============================] - 0s 73us/step - loss: 0.1305 - accuracy: 0.9382 - val_loss: 0.8365 - val_accuracy: 0.9128
Epoch 781/1000
453/453 [==============================] - 0s 70us/step - loss: 0.1303 - accuracy: 0.9382 - val_loss: 0.9428 - val_accuracy: 0.9128
Epoch 782/1000
453/453 [==============================] - 0s 69us/step - loss: 0.1312 - accuracy: 0.9382 - val_loss: 0.8318 - val_accuracy: 0.9128
Epoch 783/1000
453/453 [==============================] - 0s 72us/step - loss: 0.1326 - accuracy: 0.9360 - val_loss: 0.8310 - val_accuracy: 0.9128
Epoch 784/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1327 - accuracy: 0.9360 - val_loss: 0.8331 - val_accuracy: 0.9128
Epoch 785/1000
453/453 [==============================] - 0s 69us/step - loss: 0.1386 - accuracy: 0.9382 - val_loss: 0.9554 - val_accuracy: 0.9128
Epoch 786/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1370 - accuracy: 0.9338 - val_loss: 0

Epoch 836/1000
453/453 [==============================] - 0s 72us/step - loss: 1.2217 - accuracy: 0.8742 - val_loss: 0.9059 - val_accuracy: 0.8974
Epoch 837/1000
453/453 [==============================] - 0s 65us/step - loss: 1.5782 - accuracy: 0.8256 - val_loss: 1.0984 - val_accuracy: 0.8718
Epoch 838/1000
453/453 [==============================] - 0s 70us/step - loss: 0.7333 - accuracy: 0.8874 - val_loss: 1.6492 - val_accuracy: 0.8872
Epoch 839/1000
453/453 [==============================] - 0s 75us/step - loss: 0.4080 - accuracy: 0.9183 - val_loss: 0.9063 - val_accuracy: 0.8974
Epoch 840/1000
453/453 [==============================] - 0s 67us/step - loss: 0.2675 - accuracy: 0.9227 - val_loss: 1.7540 - val_accuracy: 0.8821
Epoch 841/1000
453/453 [==============================] - 0s 66us/step - loss: 0.3022 - accuracy: 0.9183 - val_loss: 0.9426 - val_accuracy: 0.8821
Epoch 842/1000
453/453 [==============================] - 0s 75us/step - loss: 0.3402 - accuracy: 0.9095 - val_loss: 1

Epoch 892/1000
453/453 [==============================] - 0s 66us/step - loss: 0.1316 - accuracy: 0.9338 - val_loss: 1.0570 - val_accuracy: 0.9128
Epoch 893/1000
453/453 [==============================] - 0s 75us/step - loss: 0.1349 - accuracy: 0.9382 - val_loss: 1.0495 - val_accuracy: 0.9128
Epoch 894/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1323 - accuracy: 0.9382 - val_loss: 1.0516 - val_accuracy: 0.9128
Epoch 895/1000
453/453 [==============================] - 0s 72us/step - loss: 0.1319 - accuracy: 0.9382 - val_loss: 1.0548 - val_accuracy: 0.9128
Epoch 896/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1336 - accuracy: 0.9382 - val_loss: 1.0341 - val_accuracy: 0.9128
Epoch 897/1000
453/453 [==============================] - 0s 73us/step - loss: 0.1338 - accuracy: 0.9382 - val_loss: 1.0631 - val_accuracy: 0.9128
Epoch 898/1000
453/453 [==============================] - 0s 69us/step - loss: 0.1288 - accuracy: 0.9382 - val_loss: 1

In [56]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 93.45%


In [30]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [31]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [32]:
y_prob2 = df_proba2[df_proba2['phage']=='p0017SkpresabsSTCC_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[7.07914900e-07, 1.54166300e-07, 9.99999170e-01],
       [1.14630350e-09, 9.99999900e-01, 9.94400600e-08],
       [2.09464310e-01, 7.90535700e-01, 9.97526500e-10],
       [1.58016130e-03, 9.98380400e-01, 3.94244500e-05],
       [4.17502370e-02, 9.58249700e-01, 2.64531800e-11],
       [3.02567660e-01, 6.97432340e-01, 8.08262600e-13],
       [8.42038400e-03, 9.91579650e-01, 9.75796800e-09],
       [9.50591800e-04, 9.99049370e-01, 8.57430300e-12],
       [1.00000000e+00, 1.69704410e-24, 2.81239230e-21],
       [7.07914900e-07, 1.54166300e-07, 9.99999170e-01],
       [1.62624730e-01, 8.37375300e-01, 4.65597140e-11],
       [6.76157000e-01, 3.23843030e-01, 2.47446420e-11],
       [4.44197580e-06, 9.70133400e-06, 9.99985800e-01],
       [4.44197580e-06, 9.70133400e-06, 9.99985800e-01],
       [3.26104160e-08, 1.00000000e+00, 5.69002900e-09],
       [7.07914900e-07, 1.54166300e-07, 9.99999170e-01],
       [9.99999900e-01, 6.89115250e-08, 1.43344350e-17],
       [1.62624730e-01, 8.37375

In [33]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.9613609467455623

In [34]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.9613609467455623

In [35]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [36]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [37]:
dat3

,0,test
0,NRS249,1
1,NRS172,1
2,NRS209,2
3,NRS108,1
4,NRS209,2
...,...,...
190,NRS209,2
191,NRS110,2
192,NRS255,1
193,NRS175,1


In [38]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [44]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [45]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 271us/step - loss: 3.9185 - accuracy: 0.4305 - val_loss: 1.7662 - val_accuracy: 0.4667
Epoch 2/1000
453/453 [==============================] - 0s 96us/step - loss: 1.5113 - accuracy: 0.5453 - val_loss: 0.8505 - val_accuracy: 0.6923
Epoch 3/1000
453/453 [==============================] - 0s 76us/step - loss: 0.8163 - accuracy: 0.6909 - val_loss: 1.1008 - val_accuracy: 0.6513
Epoch 4/1000
453/453 [==============================] - 0s 118us/step - loss: 0.7155 - accuracy: 0.6909 - val_loss: 0.6893 - val_accuracy: 0.7590
Epoch 5/1000
453/453 [==============================] - 0s 109us/step - loss: 0.7139 - accuracy: 0.6821 - val_loss: 0.8753 - val_accuracy: 0.7179
Epoch 6/1000
453/453 [==============================] - 0s 102us/step - loss: 0.6092 - accuracy: 0.7483 - val_loss: 0.5706 - val_accuracy: 0.7231
Epoch 7/1000
453/453 [==============================] - 0s 107us/step - loss: 0.

Epoch 112/1000
453/453 [==============================] - 0s 59us/step - loss: 0.2983 - accuracy: 0.8918 - val_loss: 0.2674 - val_accuracy: 0.8667
Epoch 113/1000
453/453 [==============================] - 0s 64us/step - loss: 0.3688 - accuracy: 0.8830 - val_loss: 0.2711 - val_accuracy: 0.8667
Epoch 114/1000
453/453 [==============================] - 0s 55us/step - loss: 0.2435 - accuracy: 0.9095 - val_loss: 0.2408 - val_accuracy: 0.8974
Epoch 115/1000
453/453 [==============================] - 0s 56us/step - loss: 0.2300 - accuracy: 0.9249 - val_loss: 0.2394 - val_accuracy: 0.8974
Epoch 116/1000
453/453 [==============================] - 0s 60us/step - loss: 0.2371 - accuracy: 0.9117 - val_loss: 0.2358 - val_accuracy: 0.8974
Epoch 117/1000
453/453 [==============================] - 0s 58us/step - loss: 0.2330 - accuracy: 0.9205 - val_loss: 0.2471 - val_accuracy: 0.8923
Epoch 118/1000
453/453 [==============================] - 0s 72us/step - loss: 0.2316 - accuracy: 0.9161 - val_loss: 0

Epoch 168/1000
453/453 [==============================] - 0s 57us/step - loss: 0.2070 - accuracy: 0.9183 - val_loss: 0.2331 - val_accuracy: 0.8821
Epoch 169/1000
453/453 [==============================] - 0s 61us/step - loss: 0.2134 - accuracy: 0.9139 - val_loss: 0.2259 - val_accuracy: 0.8974
Epoch 170/1000
453/453 [==============================] - 0s 55us/step - loss: 0.2051 - accuracy: 0.9249 - val_loss: 0.2215 - val_accuracy: 0.8974
Epoch 171/1000
453/453 [==============================] - 0s 56us/step - loss: 0.2024 - accuracy: 0.9205 - val_loss: 0.2221 - val_accuracy: 0.8974
Epoch 172/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2019 - accuracy: 0.9227 - val_loss: 0.2229 - val_accuracy: 0.8974
Epoch 173/1000
453/453 [==============================] - 0s 83us/step - loss: 0.2079 - accuracy: 0.9205 - val_loss: 0.2226 - val_accuracy: 0.8974
Epoch 174/1000
453/453 [==============================] - 0s 75us/step - loss: 0.2037 - accuracy: 0.9249 - val_loss: 

Epoch 224/1000
453/453 [==============================] - 0s 88us/step - loss: 0.1845 - accuracy: 0.9205 - val_loss: 0.2055 - val_accuracy: 0.8974
Epoch 225/1000
453/453 [==============================] - 0s 106us/step - loss: 0.1862 - accuracy: 0.9161 - val_loss: 0.2069 - val_accuracy: 0.9026
Epoch 226/1000
453/453 [==============================] - 0s 90us/step - loss: 0.1861 - accuracy: 0.9316 - val_loss: 0.2112 - val_accuracy: 0.8974
Epoch 227/1000
453/453 [==============================] - 0s 68us/step - loss: 0.1849 - accuracy: 0.9227 - val_loss: 0.2069 - val_accuracy: 0.8974
Epoch 228/1000
453/453 [==============================] - 0s 63us/step - loss: 0.1836 - accuracy: 0.9249 - val_loss: 0.2061 - val_accuracy: 0.9026
Epoch 229/1000
453/453 [==============================] - 0s 61us/step - loss: 0.1856 - accuracy: 0.9227 - val_loss: 0.2084 - val_accuracy: 0.8974
Epoch 230/1000
453/453 [==============================] - 0s 63us/step - loss: 0.1832 - accuracy: 0.9272 - val_loss: 

Epoch 335/1000
453/453 [==============================] - 0s 70us/step - loss: 0.1623 - accuracy: 0.9294 - val_loss: 0.1931 - val_accuracy: 0.9128
Epoch 336/1000
453/453 [==============================] - 0s 65us/step - loss: 0.1616 - accuracy: 0.9294 - val_loss: 0.1927 - val_accuracy: 0.9128
Epoch 337/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1624 - accuracy: 0.9294 - val_loss: 0.1931 - val_accuracy: 0.9128
Epoch 338/1000
453/453 [==============================] - 0s 63us/step - loss: 0.1631 - accuracy: 0.9294 - val_loss: 0.1943 - val_accuracy: 0.9128
Epoch 339/1000
453/453 [==============================] - 0s 69us/step - loss: 0.1640 - accuracy: 0.9316 - val_loss: 0.1924 - val_accuracy: 0.9128
Epoch 340/1000
453/453 [==============================] - 0s 63us/step - loss: 0.1646 - accuracy: 0.9294 - val_loss: 0.1951 - val_accuracy: 0.9128
Epoch 341/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1644 - accuracy: 0.9294 - val_loss: 0

...

Epoch 613/1000
453/453 [==============================] - 0s 69us/step - loss: 0.1473 - accuracy: 0.9272 - val_loss: 0.2013 - val_accuracy: 0.9179
Epoch 614/1000
453/453 [==============================] - 0s 70us/step - loss: 0.1527 - accuracy: 0.9294 - val_loss: 0.2058 - val_accuracy: 0.9179
Epoch 615/1000
453/453 [==============================] - 0s 72us/step - loss: 0.1529 - accuracy: 0.9294 - val_loss: 0.2041 - val_accuracy: 0.9179
Epoch 616/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1632 - accuracy: 0.9205 - val_loss: 0.2005 - val_accuracy: 0.9128
Epoch 617/1000
453/453 [==============================] - 0s 72us/step - loss: 0.1516 - accuracy: 0.9316 - val_loss: 0.1976 - val_accuracy: 0.9128
Epoch 618/1000
453/453 [==============================] - 0s 66us/step - loss: 0.1491 - accuracy: 0.9316 - val_loss: 0.2017 - val_accuracy: 0.9179
Epoch 619/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1510 - accuracy: 0.9272 - val_loss: 0

Epoch 669/1000
453/453 [==============================] - 0s 72us/step - loss: 0.1452 - accuracy: 0.9316 - val_loss: 0.2054 - val_accuracy: 0.9179
Epoch 670/1000
453/453 [==============================] - 0s 70us/step - loss: 0.1545 - accuracy: 0.9338 - val_loss: 0.2013 - val_accuracy: 0.9179
Epoch 671/1000
453/453 [==============================] - 0s 75us/step - loss: 0.1466 - accuracy: 0.9294 - val_loss: 0.2087 - val_accuracy: 0.9128
Epoch 672/1000
453/453 [==============================] - 0s 74us/step - loss: 0.1468 - accuracy: 0.9316 - val_loss: 0.2095 - val_accuracy: 0.9128
Epoch 673/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1506 - accuracy: 0.9294 - val_loss: 0.2129 - val_accuracy: 0.9128
Epoch 674/1000
453/453 [==============================] - 0s 68us/step - loss: 0.1503 - accuracy: 0.9294 - val_loss: 0.2045 - val_accuracy: 0.9128
Epoch 675/1000
453/453 [==============================] - 0s 68us/step - loss: 0.1490 - accuracy: 0.9316 - val_loss: 0

Epoch 725/1000
453/453 [==============================] - 0s 72us/step - loss: 0.1484 - accuracy: 0.9316 - val_loss: 0.2052 - val_accuracy: 0.9179
Epoch 726/1000
453/453 [==============================] - ETA: 0s - loss: 0.0380 - accuracy: 0.96 - 0s 76us/step - loss: 0.1506 - accuracy: 0.9338 - val_loss: 0.2019 - val_accuracy: 0.9179
Epoch 727/1000
453/453 [==============================] - 0s 73us/step - loss: 0.1485 - accuracy: 0.9205 - val_loss: 0.2029 - val_accuracy: 0.9179
Epoch 728/1000
453/453 [==============================] - 0s 72us/step - loss: 0.1491 - accuracy: 0.9338 - val_loss: 0.2000 - val_accuracy: 0.9179
Epoch 729/1000
453/453 [==============================] - 0s 65us/step - loss: 0.1450 - accuracy: 0.9316 - val_loss: 0.2039 - val_accuracy: 0.9179
Epoch 730/1000
453/453 [==============================] - 0s 65us/step - loss: 0.1464 - accuracy: 0.9316 - val_loss: 0.2004 - val_accuracy: 0.9179
Epoch 731/1000
453/453 [==============================] - 0s 72us/step - los

Epoch 836/1000
453/453 [==============================] - 0s 68us/step - loss: 0.1447 - accuracy: 0.9294 - val_loss: 0.2077 - val_accuracy: 0.9128
Epoch 837/1000
453/453 [==============================] - 0s 65us/step - loss: 0.1500 - accuracy: 0.9272 - val_loss: 0.2110 - val_accuracy: 0.9179
Epoch 838/1000
453/453 [==============================] - 0s 69us/step - loss: 0.1491 - accuracy: 0.9338 - val_loss: 0.2143 - val_accuracy: 0.9128
Epoch 839/1000
453/453 [==============================] - 0s 65us/step - loss: 0.1484 - accuracy: 0.9183 - val_loss: 0.2147 - val_accuracy: 0.9128
Epoch 840/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1445 - accuracy: 0.9338 - val_loss: 0.2109 - val_accuracy: 0.9128
Epoch 841/1000
453/453 [==============================] - 0s 69us/step - loss: 0.1466 - accuracy: 0.9360 - val_loss: 0.2121 - val_accuracy: 0.9128
Epoch 842/1000
453/453 [==============================] - 0s 68us/step - loss: 0.1443 - accuracy: 0.9316 - val_loss: 0

Epoch 892/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1577 - accuracy: 0.9294 - val_loss: 0.2148 - val_accuracy: 0.8974
Epoch 893/1000
453/453 [==============================] - 0s 79us/step - loss: 0.1540 - accuracy: 0.9316 - val_loss: 0.2085 - val_accuracy: 0.9077
Epoch 894/1000
453/453 [==============================] - 0s 73us/step - loss: 0.1512 - accuracy: 0.9316 - val_loss: 0.2116 - val_accuracy: 0.9128
Epoch 895/1000
453/453 [==============================] - 0s 68us/step - loss: 0.1523 - accuracy: 0.9360 - val_loss: 0.2039 - val_accuracy: 0.9128
Epoch 896/1000
453/453 [==============================] - 0s 66us/step - loss: 0.1510 - accuracy: 0.9316 - val_loss: 0.2133 - val_accuracy: 0.9128
Epoch 897/1000
453/453 [==============================] - 0s 74us/step - loss: 0.1463 - accuracy: 0.9316 - val_loss: 0.2073 - val_accuracy: 0.9077
Epoch 898/1000
453/453 [==============================] - 0s 72us/step - loss: 0.1515 - accuracy: 0.9249 - val_loss: 0

In [70]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

195/195 [==============================] - 0s 51us/step
over-sampling test accuracy: 91.28%


In [47]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 2, 0, 2, 0, 0, 1, 2,
       1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 2, 2, 1, 0, 2, 1, 0, 0,
       2, 2, 2, 2, 2, 1, 2, 2, 1, 0, 2, 2, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 2, 2, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 2, 1, 0, 2, 0, 2, 0, 1, 2,
       1, 2, 0, 0, 2, 0, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 0, 2, 1, 2, 0, 1,
       2, 0, 1, 0, 1, 1, 2, 2, 1, 1, 0, 0, 2, 1, 1, 0, 1, 0, 2, 2, 0, 1,
       2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 2, 1, 1, 1, 2, 0, 2, 2, 0, 2, 1, 0,
       1, 2, 0, 1, 1, 0, 0, 2, 0, 0, 0, 1, 2, 2, 0, 1, 1, 2, 0, 1, 2, 2,
       0, 2, 0, 0, 2, 0, 2, 1, 1, 0, 1, 0, 0, 2, 2, 2, 1, 1, 1])

In [48]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,NRS249,1,1
1,NRS172,1,1
2,NRS209,2,2
3,NRS108,1,1
4,NRS209,2,2
...,...,...,...
190,NRS209,2,2
191,NRS110,2,2
192,NRS255,1,1
193,NRS175,1,1


In [49]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [50]:
dat_proba3

,0,1,2
0,9.744758e-02,9.025524e-01,1.978138e-12
1,6.600096e-24,1.000000e+00,2.792151e-17
2,4.103175e-07,3.448591e-08,9.999995e-01
3,4.459796e-13,1.000000e+00,5.674695e-12
4,4.103175e-07,3.448591e-08,9.999995e-01
...,...,...,...
190,4.103175e-07,3.448591e-08,9.999995e-01
191,9.831741e-07,1.349565e-07,9.999989e-01
192,3.136863e-29,1.000000e+00,2.076573e-18
193,3.381405e-01,6.618510e-01,8.425885e-06


In [51]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [52]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p17sST.csv", index = False,
         header=None)

In [74]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 189us/step - loss: 0.1574 - accuracy: 0.8985 - val_loss: 0.1762 - val_accuracy: 0.9128
Epoch 2/1000
453/453 [==============================] - 0s 128us/step - loss: 0.1530 - accuracy: 0.9272 - val_loss: 0.2802 - val_accuracy: 0.8872
Epoch 3/1000
453/453 [==============================] - 0s 102us/step - loss: 0.1745 - accuracy: 0.9249 - val_loss: 1.1958 - val_accuracy: 0.8359
Epoch 4/1000
453/453 [==============================] - 0s 104us/step - loss: 0.6598 - accuracy: 0.8808 - val_loss: 0.7888 - val_accuracy: 0.8667
Epoch 5/1000
453/453 [==============================] - 0s 83us/step - loss: 0.2460 - accuracy: 0.8587 - val_loss: 0.1865 - val_accuracy: 0.9128
Epoch 6/1000
453/453 [==============================] - 0s 112us/step - loss: 0.1655 - accuracy: 0.9316 - val_loss: 0.1842 - val_accuracy: 0.9128
Epoch 7/1000
453/453 [==============================] - 0s 85us/step - loss: 0.

Epoch 57/1000
453/453 [==============================] - 0s 101us/step - loss: 0.1376 - accuracy: 0.9382 - val_loss: 0.1872 - val_accuracy: 0.9179
Epoch 58/1000
453/453 [==============================] - 0s 83us/step - loss: 0.1363 - accuracy: 0.9382 - val_loss: 0.1892 - val_accuracy: 0.9128
Epoch 59/1000
453/453 [==============================] - 0s 86us/step - loss: 0.1366 - accuracy: 0.9382 - val_loss: 0.1838 - val_accuracy: 0.9128
Epoch 60/1000
453/453 [==============================] - 0s 89us/step - loss: 0.1389 - accuracy: 0.9360 - val_loss: 0.1793 - val_accuracy: 0.9128
Epoch 61/1000
453/453 [==============================] - 0s 94us/step - loss: 0.1408 - accuracy: 0.9360 - val_loss: 0.1836 - val_accuracy: 0.9077
Epoch 62/1000
453/453 [==============================] - 0s 123us/step - loss: 0.1402 - accuracy: 0.9382 - val_loss: 0.1855 - val_accuracy: 0.9128
Epoch 63/1000
453/453 [==============================] - 0s 293us/step - loss: 0.1385 - accuracy: 0.9382 - val_loss: 0.180

453/453 [==============================] - 0s 109us/step - loss: 0.1399 - accuracy: 0.9316 - val_loss: 0.1830 - val_accuracy: 0.9179
Epoch 168/1000
453/453 [==============================] - 0s 105us/step - loss: 0.1362 - accuracy: 0.9382 - val_loss: 0.1828 - val_accuracy: 0.9179
Epoch 169/1000
453/453 [==============================] - 0s 102us/step - loss: 0.1350 - accuracy: 0.9382 - val_loss: 0.1844 - val_accuracy: 0.9128
Epoch 170/1000
453/453 [==============================] - 0s 101us/step - loss: 0.1367 - accuracy: 0.9382 - val_loss: 0.1859 - val_accuracy: 0.9128
Epoch 171/1000
453/453 [==============================] - 0s 96us/step - loss: 0.1361 - accuracy: 0.9382 - val_loss: 0.1825 - val_accuracy: 0.9179
Epoch 172/1000
453/453 [==============================] - 0s 96us/step - loss: 0.1361 - accuracy: 0.9382 - val_loss: 0.1791 - val_accuracy: 0.9179
Epoch 173/1000
453/453 [==============================] - 0s 92us/step - loss: 0.1355 - accuracy: 0.9382 - val_loss: 0.1768 - val

453/453 [==============================] - 0s 82us/step - loss: 0.1362 - accuracy: 0.9382 - val_loss: 0.1872 - val_accuracy: 0.9179
Epoch 278/1000
453/453 [==============================] - 0s 94us/step - loss: 0.1396 - accuracy: 0.9338 - val_loss: 0.1807 - val_accuracy: 0.9128
Epoch 279/1000
453/453 [==============================] - 0s 111us/step - loss: 0.1380 - accuracy: 0.9382 - val_loss: 0.1902 - val_accuracy: 0.9128
Epoch 280/1000
453/453 [==============================] - 0s 116us/step - loss: 0.1356 - accuracy: 0.9382 - val_loss: 0.1859 - val_accuracy: 0.9179
Epoch 281/1000
453/453 [==============================] - 0s 102us/step - loss: 0.1399 - accuracy: 0.9382 - val_loss: 0.1827 - val_accuracy: 0.9128
Epoch 282/1000
453/453 [==============================] - 0s 80us/step - loss: 0.1387 - accuracy: 0.9382 - val_loss: 0.1864 - val_accuracy: 0.9128
Epoch 283/1000
453/453 [==============================] - 0s 79us/step - loss: 0.1339 - accuracy: 0.9382 - val_loss: 0.1860 - val_

Epoch 333/1000
453/453 [==============================] - 0s 127us/step - loss: 0.1360 - accuracy: 0.9338 - val_loss: 0.1942 - val_accuracy: 0.9128
Epoch 334/1000
453/453 [==============================] - 0s 135us/step - loss: 0.1371 - accuracy: 0.9382 - val_loss: 0.1865 - val_accuracy: 0.9128
Epoch 335/1000
453/453 [==============================] - 0s 149us/step - loss: 0.1435 - accuracy: 0.9272 - val_loss: 0.1852 - val_accuracy: 0.9128
Epoch 336/1000
453/453 [==============================] - 0s 243us/step - loss: 0.1432 - accuracy: 0.9360 - val_loss: 0.1965 - val_accuracy: 0.9128
Epoch 337/1000
453/453 [==============================] - 0s 296us/step - loss: 0.1427 - accuracy: 0.9382 - val_loss: 0.1945 - val_accuracy: 0.9026
Epoch 338/1000
453/453 [==============================] - 0s 114us/step - loss: 0.1569 - accuracy: 0.9117 - val_loss: 0.1856 - val_accuracy: 0.9128
Epoch 339/1000
453/453 [==============================] - 0s 102us/step - loss: 0.1349 - accuracy: 0.9382 - val_

...

453/453 [==============================] - 0s 67us/step - loss: 0.1404 - accuracy: 0.9161 - val_loss: 0.1921 - val_accuracy: 0.9077
Epoch 777/1000
453/453 [==============================] - 0s 79us/step - loss: 0.1428 - accuracy: 0.9360 - val_loss: 0.2027 - val_accuracy: 0.9077
Epoch 778/1000
453/453 [==============================] - 0s 66us/step - loss: 0.1385 - accuracy: 0.9404 - val_loss: 0.1971 - val_accuracy: 0.9077
Epoch 779/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1330 - accuracy: 0.9382 - val_loss: 0.1984 - val_accuracy: 0.9077
Epoch 780/1000
453/453 [==============================] - 0s 72us/step - loss: 0.1367 - accuracy: 0.9382 - val_loss: 0.2031 - val_accuracy: 0.9077
Epoch 781/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1413 - accuracy: 0.9360 - val_loss: 0.1979 - val_accuracy: 0.9077
Epoch 782/1000
453/453 [==============================] - 0s 66us/step - loss: 0.1387 - accuracy: 0.9382 - val_loss: 0.1974 - val_acc

Epoch 832/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1350 - accuracy: 0.9382 - val_loss: 0.2063 - val_accuracy: 0.9077
Epoch 833/1000
453/453 [==============================] - 0s 75us/step - loss: 0.1368 - accuracy: 0.9294 - val_loss: 0.1990 - val_accuracy: 0.9128
Epoch 834/1000
453/453 [==============================] - 0s 73us/step - loss: 0.1611 - accuracy: 0.9294 - val_loss: 0.2018 - val_accuracy: 0.9077
Epoch 835/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1410 - accuracy: 0.9294 - val_loss: 0.2043 - val_accuracy: 0.9077
Epoch 836/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1341 - accuracy: 0.9382 - val_loss: 0.1970 - val_accuracy: 0.9128
Epoch 837/1000
453/453 [==============================] - 0s 74us/step - loss: 0.1339 - accuracy: 0.9382 - val_loss: 0.2023 - val_accuracy: 0.9077
Epoch 838/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1363 - accuracy: 0.9382 - val_loss: 0

Epoch 888/1000
453/453 [==============================] - 0s 77us/step - loss: 0.1338 - accuracy: 0.9382 - val_loss: 0.1979 - val_accuracy: 0.9128
Epoch 889/1000
453/453 [==============================] - 0s 68us/step - loss: 0.1357 - accuracy: 0.9382 - val_loss: 0.1989 - val_accuracy: 0.9128
Epoch 890/1000
453/453 [==============================] - 0s 66us/step - loss: 0.1377 - accuracy: 0.9382 - val_loss: 0.2059 - val_accuracy: 0.9128
Epoch 891/1000
453/453 [==============================] - 0s 73us/step - loss: 0.1400 - accuracy: 0.9316 - val_loss: 0.1990 - val_accuracy: 0.9128
Epoch 892/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1416 - accuracy: 0.9382 - val_loss: 0.2030 - val_accuracy: 0.9128
Epoch 893/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1381 - accuracy: 0.9205 - val_loss: 0.1989 - val_accuracy: 0.9128
Epoch 894/1000
453/453 [==============================] - 0s 74us/step - loss: 0.1376 - accuracy: 0.9382 - val_loss: 0

Epoch 944/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1312 - accuracy: 0.9382 - val_loss: 0.1981 - val_accuracy: 0.9128
Epoch 945/1000
453/453 [==============================] - 0s 78us/step - loss: 0.1362 - accuracy: 0.9382 - val_loss: 0.1898 - val_accuracy: 0.9128
Epoch 946/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1381 - accuracy: 0.9382 - val_loss: 0.1990 - val_accuracy: 0.9128
Epoch 947/1000
453/453 [==============================] - 0s 71us/step - loss: 0.1339 - accuracy: 0.9382 - val_loss: 0.2020 - val_accuracy: 0.9128
Epoch 948/1000
453/453 [==============================] - 0s 69us/step - loss: 0.1335 - accuracy: 0.9382 - val_loss: 0.2112 - val_accuracy: 0.9128
Epoch 949/1000
453/453 [==============================] - 0s 67us/step - loss: 0.1406 - accuracy: 0.9382 - val_loss: 0.2209 - val_accuracy: 0.9128
Epoch 950/1000
453/453 [==============================] - 0s 69us/step - loss: 0.1518 - accuracy: 0.9029 - val_loss: 0

Epoch 1000/1000
453/453 [==============================] - 0s 70us/step - loss: 0.1370 - accuracy: 0.9382 - val_loss: 0.1910 - val_accuracy: 0.9128


In [75]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 93.42%


In [39]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [40]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [41]:
y_prob3 = df_proba3[df_proba3['phage']=='p0017SkpresabsSTCC_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[9.74475800e-02, 9.02552370e-01, 1.97813840e-12],
       [6.60009600e-24, 1.00000000e+00, 2.79215150e-17],
       [4.10317480e-07, 3.44859100e-08, 9.99999500e-01],
       [4.45979570e-13, 1.00000000e+00, 5.67469500e-12],
       [4.10317480e-07, 3.44859100e-08, 9.99999500e-01],
       [3.64789130e-01, 6.35200500e-01, 1.02883620e-05],
       [2.56565660e-01, 7.43432160e-01, 2.22473770e-06],
       [9.74475800e-02, 9.02552370e-01, 1.97813840e-12],
       [2.20254320e-01, 7.79745700e-01, 3.13330700e-12],
       [6.60009600e-24, 1.00000000e+00, 2.79215150e-17],
       [9.99999900e-01, 1.70347970e-07, 3.49707730e-20],
       [1.00000000e+00, 1.61197490e-09, 5.82585900e-22],
       [6.60009600e-24, 1.00000000e+00, 2.79215150e-17],
       [3.13686320e-29, 1.00000000e+00, 2.07657300e-18],
       [9.39403500e-01, 6.05965100e-02, 5.26804900e-12],
       [9.83174100e-07, 1.34956450e-07, 9.99998900e-01],
       [8.31748550e-01, 1.68250130e-01, 1.39423880e-06],
       [9.83174100e-07, 1.34956

In [42]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.9813806706114399

In [43]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.9813806706114399

In [44]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [45]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [46]:
dat4

,0,test
0,NRS241,1
1,BCH-SA-01,1
2,NRS219,1
3,NRS209,2
4,NRS001,1
...,...,...
190,GA15,1
191,NRS246,1
192,115,1
193,312,1


In [47]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [48]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [49]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 476us/step - loss: 3.6781 - accuracy: 0.4592 - val_loss: 1.5802 - val_accuracy: 0.5282
Epoch 2/1000
453/453 [==============================] - 0s 136us/step - loss: 0.8775 - accuracy: 0.6115 - val_loss: 0.6155 - val_accuracy: 0.7641
Epoch 3/1000
453/453 [==============================] - 0s 175us/step - loss: 0.6924 - accuracy: 0.6755 - val_loss: 0.5585 - val_accuracy: 0.8154
Epoch 4/1000
453/453 [==============================] - 0s 135us/step - loss: 0.5719 - accuracy: 0.8013 - val_loss: 0.5620 - val_accuracy: 0.8256
Epoch 5/1000
453/453 [==============================] - 0s 223us/step - loss: 0.5552 - accuracy: 0.7528 - val_loss: 0.4756 - val_accuracy: 0.7026
Epoch 6/1000
453/453 [==============================] - 0s 159us/step - loss: 0.4517 - accuracy: 0.8013 - val_loss: 0.4072 - val_accuracy: 0.8256
Epoch 7/1000
453/453 [==============================] - 0s 141us/step - loss: 

Epoch 112/1000
453/453 [==============================] - 0s 131us/step - loss: 0.2123 - accuracy: 0.8985 - val_loss: 0.2472 - val_accuracy: 0.8974
Epoch 113/1000
453/453 [==============================] - 0s 141us/step - loss: 0.2111 - accuracy: 0.9007 - val_loss: 0.2621 - val_accuracy: 0.8923
Epoch 114/1000
453/453 [==============================] - 0s 134us/step - loss: 0.2401 - accuracy: 0.8940 - val_loss: 0.3374 - val_accuracy: 0.8205
Epoch 115/1000
453/453 [==============================] - 0s 102us/step - loss: 0.3911 - accuracy: 0.8874 - val_loss: 0.8058 - val_accuracy: 0.8154
Epoch 116/1000
453/453 [==============================] - 0s 115us/step - loss: 0.3638 - accuracy: 0.8918 - val_loss: 0.2476 - val_accuracy: 0.9026
Epoch 117/1000
453/453 [==============================] - 0s 105us/step - loss: 0.2069 - accuracy: 0.9227 - val_loss: 0.2460 - val_accuracy: 0.8974
Epoch 118/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2034 - accuracy: 0.9205 - val_

453/453 [==============================] - 0s 100us/step - loss: 0.1776 - accuracy: 0.9249 - val_loss: 0.2293 - val_accuracy: 0.8872
Epoch 223/1000
453/453 [==============================] - 0s 109us/step - loss: 0.1785 - accuracy: 0.9161 - val_loss: 0.2291 - val_accuracy: 0.8821
Epoch 224/1000
453/453 [==============================] - 0s 109us/step - loss: 0.1799 - accuracy: 0.9183 - val_loss: 0.2402 - val_accuracy: 0.8718
Epoch 225/1000
453/453 [==============================] - 0s 102us/step - loss: 0.1781 - accuracy: 0.9249 - val_loss: 0.2395 - val_accuracy: 0.8974
Epoch 226/1000
453/453 [==============================] - 0s 110us/step - loss: 0.1903 - accuracy: 0.9007 - val_loss: 0.2472 - val_accuracy: 0.8359
Epoch 227/1000
453/453 [==============================] - 0s 108us/step - loss: 0.1945 - accuracy: 0.9183 - val_loss: 0.2368 - val_accuracy: 0.8769
Epoch 228/1000
453/453 [==============================] - 0s 101us/step - loss: 0.1804 - accuracy: 0.9183 - val_loss: 0.2274 - 

453/453 [==============================] - 0s 108us/step - loss: 0.1712 - accuracy: 0.9117 - val_loss: 0.2122 - val_accuracy: 0.8923
Epoch 333/1000
453/453 [==============================] - 0s 102us/step - loss: 0.1680 - accuracy: 0.9294 - val_loss: 0.2107 - val_accuracy: 0.9026
Epoch 334/1000
453/453 [==============================] - 0s 108us/step - loss: 0.1645 - accuracy: 0.9294 - val_loss: 0.2095 - val_accuracy: 0.8923
Epoch 335/1000
453/453 [==============================] - 0s 109us/step - loss: 0.1716 - accuracy: 0.9183 - val_loss: 0.2100 - val_accuracy: 0.8923
Epoch 336/1000
453/453 [==============================] - 0s 107us/step - loss: 0.1693 - accuracy: 0.9227 - val_loss: 0.2075 - val_accuracy: 0.9026
Epoch 337/1000
453/453 [==============================] - 0s 108us/step - loss: 0.1715 - accuracy: 0.9227 - val_loss: 0.2073 - val_accuracy: 0.8974
Epoch 338/1000
453/453 [==============================] - 0s 109us/step - loss: 0.1646 - accuracy: 0.9272 - val_loss: 0.2053 - 

453/453 [==============================] - 0s 102us/step - loss: 0.1583 - accuracy: 0.9338 - val_loss: 0.1882 - val_accuracy: 0.9282
Epoch 443/1000
453/453 [==============================] - 0s 105us/step - loss: 0.1582 - accuracy: 0.9316 - val_loss: 0.2038 - val_accuracy: 0.8821
Epoch 444/1000
453/453 [==============================] - 0s 102us/step - loss: 0.1684 - accuracy: 0.9161 - val_loss: 0.1889 - val_accuracy: 0.9231
Epoch 445/1000
453/453 [==============================] - 0s 101us/step - loss: 0.1549 - accuracy: 0.9294 - val_loss: 0.1859 - val_accuracy: 0.9282
Epoch 446/1000
453/453 [==============================] - 0s 101us/step - loss: 0.1575 - accuracy: 0.9294 - val_loss: 0.1909 - val_accuracy: 0.9179
Epoch 447/1000
453/453 [==============================] - 0s 101us/step - loss: 0.1600 - accuracy: 0.9205 - val_loss: 0.1972 - val_accuracy: 0.8872
Epoch 448/1000
453/453 [==============================] - 0s 111us/step - loss: 0.1536 - accuracy: 0.9272 - val_loss: 0.2071 - 

453/453 [==============================] - 0s 101us/step - loss: 0.1550 - accuracy: 0.9183 - val_loss: 0.1831 - val_accuracy: 0.9282
Epoch 553/1000
453/453 [==============================] - 0s 110us/step - loss: 0.1605 - accuracy: 0.9205 - val_loss: 0.1909 - val_accuracy: 0.9333
Epoch 554/1000
453/453 [==============================] - 0s 104us/step - loss: 0.1572 - accuracy: 0.9294 - val_loss: 0.1914 - val_accuracy: 0.9231
Epoch 555/1000
453/453 [==============================] - 0s 112us/step - loss: 0.1521 - accuracy: 0.9294 - val_loss: 0.1773 - val_accuracy: 0.9231
Epoch 556/1000
453/453 [==============================] - 0s 101us/step - loss: 0.1536 - accuracy: 0.9316 - val_loss: 0.1795 - val_accuracy: 0.9282
Epoch 557/1000
453/453 [==============================] - 0s 109us/step - loss: 0.1558 - accuracy: 0.9294 - val_loss: 0.1810 - val_accuracy: 0.9231
Epoch 558/1000
453/453 [==============================] - 0s 111us/step - loss: 0.1489 - accuracy: 0.9294 - val_loss: 0.1783 - 

453/453 [==============================] - 0s 103us/step - loss: 0.1518 - accuracy: 0.9272 - val_loss: 0.1880 - val_accuracy: 0.8821
Epoch 663/1000
453/453 [==============================] - 0s 108us/step - loss: 0.1628 - accuracy: 0.9161 - val_loss: 0.1831 - val_accuracy: 0.9282
Epoch 664/1000
453/453 [==============================] - 0s 103us/step - loss: 0.1573 - accuracy: 0.9316 - val_loss: 0.1700 - val_accuracy: 0.9282
Epoch 665/1000
453/453 [==============================] - 0s 102us/step - loss: 0.1504 - accuracy: 0.9316 - val_loss: 0.2011 - val_accuracy: 0.8769
Epoch 666/1000
453/453 [==============================] - 0s 100us/step - loss: 0.1549 - accuracy: 0.9316 - val_loss: 0.1804 - val_accuracy: 0.9333
Epoch 667/1000
453/453 [==============================] - 0s 99us/step - loss: 0.1538 - accuracy: 0.9227 - val_loss: 0.1760 - val_accuracy: 0.9385
Epoch 668/1000
453/453 [==============================] - 0s 112us/step - loss: 0.1568 - accuracy: 0.9183 - val_loss: 0.1723 - v

Epoch 772/1000
453/453 [==============================] - 0s 109us/step - loss: 0.1447 - accuracy: 0.9338 - val_loss: 0.1652 - val_accuracy: 0.9385
Epoch 773/1000
453/453 [==============================] - 0s 113us/step - loss: 0.1514 - accuracy: 0.9338 - val_loss: 0.1679 - val_accuracy: 0.9282
Epoch 774/1000
453/453 [==============================] - 0s 103us/step - loss: 0.1864 - accuracy: 0.9183 - val_loss: 0.4776 - val_accuracy: 0.9026
Epoch 775/1000
453/453 [==============================] - 0s 103us/step - loss: 0.4415 - accuracy: 0.8940 - val_loss: 0.2413 - val_accuracy: 0.9179
Epoch 776/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2718 - accuracy: 0.8985 - val_loss: 0.2853 - val_accuracy: 0.9026
Epoch 777/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2641 - accuracy: 0.9161 - val_loss: 0.2171 - val_accuracy: 0.9179
Epoch 778/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2311 - accuracy: 0.9117 - val_

453/453 [==============================] - 0s 100us/step - loss: 0.1701 - accuracy: 0.9095 - val_loss: 0.1649 - val_accuracy: 0.9333
Epoch 828/1000
453/453 [==============================] - 0s 113us/step - loss: 0.1516 - accuracy: 0.9249 - val_loss: 0.1703 - val_accuracy: 0.9333
Epoch 829/1000
453/453 [==============================] - 0s 101us/step - loss: 0.1519 - accuracy: 0.9316 - val_loss: 0.1626 - val_accuracy: 0.9385
Epoch 830/1000
453/453 [==============================] - 0s 106us/step - loss: 0.1501 - accuracy: 0.9338 - val_loss: 0.1744 - val_accuracy: 0.9026
Epoch 831/1000
453/453 [==============================] - 0s 109us/step - loss: 0.1492 - accuracy: 0.9227 - val_loss: 0.1597 - val_accuracy: 0.9385
Epoch 832/1000
453/453 [==============================] - 0s 108us/step - loss: 0.1403 - accuracy: 0.9338 - val_loss: 0.1730 - val_accuracy: 0.9333
Epoch 833/1000
453/453 [==============================] - 0s 110us/step - loss: 0.1492 - accuracy: 0.9360 - val_loss: 0.1582 - 

453/453 [==============================] - 0s 101us/step - loss: 0.1529 - accuracy: 0.9139 - val_loss: 0.1704 - val_accuracy: 0.9333
Epoch 936/1000
453/453 [==============================] - 0s 108us/step - loss: 0.1581 - accuracy: 0.9249 - val_loss: 0.1610 - val_accuracy: 0.9385
Epoch 937/1000
453/453 [==============================] - 0s 102us/step - loss: 0.1882 - accuracy: 0.9073 - val_loss: 0.1801 - val_accuracy: 0.9179
Epoch 938/1000
453/453 [==============================] - 0s 109us/step - loss: 0.1530 - accuracy: 0.9249 - val_loss: 0.1756 - val_accuracy: 0.9282
Epoch 939/1000
453/453 [==============================] - 0s 110us/step - loss: 0.1481 - accuracy: 0.9338 - val_loss: 0.1628 - val_accuracy: 0.9385
Epoch 940/1000
453/453 [==============================] - 0s 101us/step - loss: 0.1419 - accuracy: 0.9316 - val_loss: 0.1604 - val_accuracy: 0.9333
Epoch 941/1000
453/453 [==============================] - 0s 113us/step - loss: 0.1558 - accuracy: 0.9161 - val_loss: 0.1651 - 

In [83]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

195/195 [==============================] - 0s 59us/step
over-sampling test accuracy: 93.33%


In [60]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([1, 1, 0, 2, 1, 2, 1, 1, 2, 2, 2, 0, 0, 0, 1, 2, 2, 2, 2, 0, 1, 1,
       0, 2, 1, 0, 1, 0, 2, 1, 2, 1, 2, 2, 2, 0, 0, 2, 0, 2, 1, 2, 1, 1,
       0, 1, 2, 2, 1, 1, 2, 0, 1, 2, 1, 2, 1, 1, 0, 1, 1, 1, 0, 1, 2, 0,
       1, 0, 0, 2, 1, 1, 2, 1, 1, 0, 2, 0, 0, 2, 1, 1, 2, 2, 2, 1, 2, 2,
       2, 2, 1, 0, 1, 0, 1, 0, 1, 2, 2, 2, 0, 0, 0, 1, 1, 2, 0, 2, 0, 0,
       2, 2, 0, 2, 2, 1, 2, 2, 2, 1, 1, 0, 2, 2, 2, 0, 0, 0, 2, 0, 0, 1,
       1, 0, 0, 1, 2, 1, 1, 2, 2, 2, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 2, 1,
       0, 0, 0, 1, 1, 1, 2, 0, 2, 0, 2, 1, 0, 2, 1, 2, 1, 0, 0, 2, 1, 0,
       1, 2, 1, 0, 1, 2, 0, 2, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0])

In [61]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,NRS241,1,1
1,BCH-SA-01,1,1
2,NRS219,1,0
3,NRS209,2,2
4,NRS001,1,1
...,...,...,...
190,GA15,1,0
191,NRS246,1,1
192,115,1,1
193,312,1,1


In [62]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [63]:
dat_proba4

,0,1,2
0,0.000026,9.999734e-01,4.102141e-07
1,0.015855,9.841448e-01,3.989326e-23
2,0.910177,8.982290e-02,4.915288e-22
3,0.000002,2.634854e-07,9.999974e-01
4,0.150429,8.495706e-01,9.282862e-21
...,...,...,...
190,0.567526,4.324737e-01,1.416410e-20
191,0.118401,8.815986e-01,5.404678e-20
192,0.000013,9.999872e-01,1.416025e-22
193,0.111514,8.884863e-01,1.433217e-37


In [64]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [65]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p17sST.csv", index = False,
         header=None)

In [87]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 177us/step - loss: 0.1364 - accuracy: 0.9338 - val_loss: 0.1622 - val_accuracy: 0.9282
Epoch 2/1000
453/453 [==============================] - 0s 150us/step - loss: 0.1376 - accuracy: 0.9382 - val_loss: 0.1616 - val_accuracy: 0.9333
Epoch 3/1000
453/453 [==============================] - 0s 144us/step - loss: 0.1356 - accuracy: 0.9382 - val_loss: 0.1655 - val_accuracy: 0.9333
Epoch 4/1000
453/453 [==============================] - 0s 129us/step - loss: 0.1411 - accuracy: 0.9360 - val_loss: 0.1687 - val_accuracy: 0.8974
Epoch 5/1000
453/453 [==============================] - 0s 137us/step - loss: 0.1427 - accuracy: 0.9272 - val_loss: 0.1577 - val_accuracy: 0.9333
Epoch 6/1000
453/453 [==============================] - 0s 149us/step - loss: 0.1352 - accuracy: 0.9382 - val_loss: 0.1692 - val_accuracy: 0.9333
Epoch 7/1000
453/453 [==============================] - 0s 137us/step - loss: 

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.110002). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


453/453 [==============================] - 0s 349us/step - loss: 0.1407 - accuracy: 0.9205 - val_loss: 0.1612 - val_accuracy: 0.9333
Epoch 77/1000
453/453 [==============================] - 0s 196us/step - loss: 0.1360 - accuracy: 0.9382 - val_loss: 0.1648 - val_accuracy: 0.9333
Epoch 78/1000
453/453 [==============================] - 0s 214us/step - loss: 0.1365 - accuracy: 0.9294 - val_loss: 0.1634 - val_accuracy: 0.9333
Epoch 79/1000
453/453 [==============================] - 0s 197us/step - loss: 0.1405 - accuracy: 0.9227 - val_loss: 0.1683 - val_accuracy: 0.8974
Epoch 80/1000
453/453 [==============================] - 0s 138us/step - loss: 0.1361 - accuracy: 0.9360 - val_loss: 0.1569 - val_accuracy: 0.9333
Epoch 81/1000
453/453 [==============================] - 0s 141us/step - loss: 0.1505 - accuracy: 0.9139 - val_loss: 0.1608 - val_accuracy: 0.9333
Epoch 82/1000
453/453 [==============================] - 0s 162us/step - loss: 0.1570 - accuracy: 0.9139 - val_loss: 0.1659 - val_ac

453/453 [==============================] - 0s 238us/step - loss: 0.1477 - accuracy: 0.9227 - val_loss: 0.1680 - val_accuracy: 0.9282
Epoch 187/1000
453/453 [==============================] - 0s 162us/step - loss: 0.1329 - accuracy: 0.9382 - val_loss: 0.1703 - val_accuracy: 0.9282
Epoch 188/1000
453/453 [==============================] - 0s 168us/step - loss: 0.1411 - accuracy: 0.9227 - val_loss: 0.1641 - val_accuracy: 0.9333
Epoch 189/1000
453/453 [==============================] - 0s 188us/step - loss: 0.1365 - accuracy: 0.9316 - val_loss: 0.1775 - val_accuracy: 0.8923
Epoch 190/1000
453/453 [==============================] - 0s 181us/step - loss: 0.1408 - accuracy: 0.9360 - val_loss: 0.1647 - val_accuracy: 0.9333
Epoch 191/1000
453/453 [==============================] - 0s 167us/step - loss: 0.1360 - accuracy: 0.9382 - val_loss: 0.1624 - val_accuracy: 0.9282
Epoch 192/1000
453/453 [==============================] - 0s 235us/step - loss: 0.1419 - accuracy: 0.9382 - val_loss: 0.1614 - 

453/453 [==============================] - 0s 171us/step - loss: 0.1385 - accuracy: 0.9360 - val_loss: 0.1665 - val_accuracy: 0.9333
Epoch 297/1000
453/453 [==============================] - 0s 133us/step - loss: 0.1389 - accuracy: 0.9382 - val_loss: 0.1680 - val_accuracy: 0.9333
Epoch 298/1000
453/453 [==============================] - 0s 137us/step - loss: 0.1453 - accuracy: 0.9382 - val_loss: 0.1616 - val_accuracy: 0.9333
Epoch 299/1000
453/453 [==============================] - 0s 194us/step - loss: 0.1384 - accuracy: 0.9294 - val_loss: 0.1622 - val_accuracy: 0.9333
Epoch 300/1000
453/453 [==============================] - 0s 125us/step - loss: 0.2939 - accuracy: 0.9029 - val_loss: 0.5974 - val_accuracy: 0.8821
Epoch 301/1000
453/453 [==============================] - 0s 117us/step - loss: 0.3303 - accuracy: 0.9117 - val_loss: 0.2728 - val_accuracy: 0.9026
Epoch 302/1000
453/453 [==============================] - 0s 133us/step - loss: 0.1924 - accuracy: 0.9139 - val_loss: 0.2018 - 

...

453/453 [==============================] - 0s 229us/step - loss: 0.1395 - accuracy: 0.9249 - val_loss: 0.1591 - val_accuracy: 0.9333
Epoch 517/1000
453/453 [==============================] - 0s 233us/step - loss: 0.1355 - accuracy: 0.9338 - val_loss: 0.1593 - val_accuracy: 0.9333
Epoch 518/1000
453/453 [==============================] - 0s 309us/step - loss: 0.1351 - accuracy: 0.9382 - val_loss: 0.1596 - val_accuracy: 0.9333
Epoch 519/1000
453/453 [==============================] - 0s 284us/step - loss: 0.1359 - accuracy: 0.9382 - val_loss: 0.1613 - val_accuracy: 0.9333
Epoch 520/1000
453/453 [==============================] - 0s 293us/step - loss: 0.1354 - accuracy: 0.9382 - val_loss: 0.1612 - val_accuracy: 0.9333
Epoch 521/1000
453/453 [==============================] - 0s 224us/step - loss: 0.1379 - accuracy: 0.9382 - val_loss: 0.1631 - val_accuracy: 0.9333
Epoch 522/1000
453/453 [==============================] - 0s 185us/step - loss: 0.1346 - accuracy: 0.9382 - val_loss: 0.1629 - 

453/453 [==============================] - 0s 129us/step - loss: 0.1369 - accuracy: 0.9382 - val_loss: 0.1613 - val_accuracy: 0.9333
Epoch 627/1000
453/453 [==============================] - 0s 129us/step - loss: 0.1321 - accuracy: 0.9382 - val_loss: 0.1588 - val_accuracy: 0.9333
Epoch 628/1000
453/453 [==============================] - 0s 126us/step - loss: 0.1345 - accuracy: 0.9382 - val_loss: 0.1590 - val_accuracy: 0.9333
Epoch 629/1000
453/453 [==============================] - 0s 135us/step - loss: 0.1313 - accuracy: 0.9360 - val_loss: 0.1589 - val_accuracy: 0.9333
Epoch 630/1000
453/453 [==============================] - 0s 135us/step - loss: 0.1373 - accuracy: 0.9360 - val_loss: 0.1707 - val_accuracy: 0.8974
Epoch 631/1000
453/453 [==============================] - 0s 136us/step - loss: 0.1466 - accuracy: 0.9316 - val_loss: 0.1687 - val_accuracy: 0.9333
Epoch 632/1000
453/453 [==============================] - 0s 132us/step - loss: 0.1387 - accuracy: 0.9227 - val_loss: 0.1599 - 

453/453 [==============================] - 0s 218us/step - loss: 0.1372 - accuracy: 0.9205 - val_loss: 0.1629 - val_accuracy: 0.9333
Epoch 737/1000
453/453 [==============================] - 0s 171us/step - loss: 0.1428 - accuracy: 0.9183 - val_loss: 0.1625 - val_accuracy: 0.9333
Epoch 738/1000
453/453 [==============================] - 0s 134us/step - loss: 0.1358 - accuracy: 0.9338 - val_loss: 0.1741 - val_accuracy: 0.8974
Epoch 739/1000
453/453 [==============================] - 0s 129us/step - loss: 0.1353 - accuracy: 0.9360 - val_loss: 0.1634 - val_accuracy: 0.9333
Epoch 740/1000
453/453 [==============================] - 0s 122us/step - loss: 0.1319 - accuracy: 0.9382 - val_loss: 0.1620 - val_accuracy: 0.9333
Epoch 741/1000
453/453 [==============================] - 0s 124us/step - loss: 0.1305 - accuracy: 0.9382 - val_loss: 0.1780 - val_accuracy: 0.8923
Epoch 742/1000
453/453 [==============================] - 0s 118us/step - loss: 0.1393 - accuracy: 0.9272 - val_loss: 0.1645 - 

453/453 [==============================] - 0s 100us/step - loss: 0.1335 - accuracy: 0.9360 - val_loss: 0.1729 - val_accuracy: 0.9333
Epoch 847/1000
453/453 [==============================] - 0s 96us/step - loss: 0.1494 - accuracy: 0.9249 - val_loss: 0.1713 - val_accuracy: 0.9333
Epoch 848/1000
453/453 [==============================] - 0s 97us/step - loss: 0.1313 - accuracy: 0.9382 - val_loss: 0.1687 - val_accuracy: 0.9333
Epoch 849/1000
453/453 [==============================] - 0s 90us/step - loss: 0.1321 - accuracy: 0.9382 - val_loss: 0.1694 - val_accuracy: 0.9333
Epoch 850/1000
453/453 [==============================] - 0s 88us/step - loss: 0.1330 - accuracy: 0.9382 - val_loss: 0.1697 - val_accuracy: 0.9333
Epoch 851/1000
453/453 [==============================] - 0s 98us/step - loss: 0.1328 - accuracy: 0.9382 - val_loss: 0.1734 - val_accuracy: 0.9333
Epoch 852/1000
453/453 [==============================] - 0s 95us/step - loss: 0.1393 - accuracy: 0.9316 - val_loss: 0.1679 - val_ac

Epoch 902/1000
453/453 [==============================] - 0s 91us/step - loss: 0.1480 - accuracy: 0.9316 - val_loss: 0.1642 - val_accuracy: 0.9333
Epoch 903/1000
453/453 [==============================] - 0s 105us/step - loss: 0.1366 - accuracy: 0.9382 - val_loss: 0.1641 - val_accuracy: 0.9333
Epoch 904/1000
453/453 [==============================] - 0s 176us/step - loss: 0.1329 - accuracy: 0.9382 - val_loss: 0.1681 - val_accuracy: 0.9333
Epoch 905/1000
453/453 [==============================] - 0s 260us/step - loss: 0.1340 - accuracy: 0.9360 - val_loss: 0.1647 - val_accuracy: 0.9333
Epoch 906/1000
453/453 [==============================] - 0s 194us/step - loss: 0.1347 - accuracy: 0.9382 - val_loss: 0.1725 - val_accuracy: 0.9282
Epoch 907/1000
453/453 [==============================] - 0s 174us/step - loss: 0.1398 - accuracy: 0.9382 - val_loss: 0.1646 - val_accuracy: 0.9333
Epoch 908/1000
453/453 [==============================] - 0s 121us/step - loss: 0.1326 - accuracy: 0.9382 - val_l

In [88]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 93.31%


In [50]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [51]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [52]:
y_prob4 = df_proba4[df_proba4['phage']=='p0017SkpresabsSTCC_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[2.62068970e-05, 9.99973400e-01, 4.10214060e-07],
       [1.58552020e-02, 9.84144800e-01, 3.98932580e-23],
       [9.10177100e-01, 8.98229000e-02, 4.91528800e-22],
       [2.33020400e-06, 2.63485400e-07, 9.99997400e-01],
       [1.50429340e-01, 8.49570630e-01, 9.28286200e-21],
       [2.33020400e-06, 2.63485400e-07, 9.99997400e-01],
       [2.09339570e-09, 9.99999640e-01, 3.35579760e-07],
       [1.33765300e-01, 8.66233050e-01, 1.69130620e-06],
       [1.02913100e-12, 4.57986320e-08, 1.00000000e+00],
       [2.33020400e-06, 2.63485400e-07, 9.99997400e-01],
       [1.02913100e-12, 4.57986320e-08, 1.00000000e+00],
       [9.99982360e-01, 2.62462340e-06, 1.50686620e-05],
       [9.95833500e-01, 4.16642400e-03, 9.37528430e-23],
       [7.97352850e-01, 2.02647210e-01, 3.61825430e-18],
       [5.70171600e-04, 9.99429760e-01, 7.79817300e-20],
       [1.02913100e-12, 4.57986320e-08, 1.00000000e+00],
       [1.02913100e-12, 4.57986320e-08, 1.00000000e+00],
       [2.33020400e-06, 2.63485

In [53]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.9864299802761342

In [54]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.9864299802761342

In [55]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.9702218934911243

In [56]:
np.std(ovos)

0.014214479688006546

In [57]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.9702218934911243

In [58]:
np.std(ovrs)

0.014214479688006546

In [89]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [90]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 91.54%


In [91]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.01057206507656773


In [92]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [93]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 93.52%


In [94]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.002274749
